In [ ]:
import pandas as pd
import numpy as np

# Create fake trip link data
routee_input_df = pd.DataFrame({
    "trip_id": np.repeat(["trip_1", "trip_2"], 3),  # two trips, 3 links each
    "kilometers": [0.5, 0.8, 1.0, 1.2, 0.7, 1.5],
    "travel_time_minutes": [1.0, 2.0, 2.5, 3.0, 1.5, 4.0],
    "grade": [0.01, -0.02, 0.0, 0.03, -0.01, 0.02],
})

In [597]:
df = pd.read_csv('/Users/yhe/github_repo/routee-transit/scripts/debug_routee_input.csv')

In [598]:
df

,trip_id,shape_id,road_id,start_lat,start_lon,end_lat,end_lon,start_timestamp,end_timestamp,kilometers,travel_time_minutes,grade
0,5167502_to_depot,to_depot_1155977,"RoadId(start=83609867, end=1595614635, key=0)",40.760641,-111.899688,40.760641,-111.899766,0 days 21:50:06,0 days 21:50:06,0.013139,0.013996,-0.0081
1,5167502_to_depot,to_depot_1155977,"RoadId(start=83633276, end=83664915, key=0)",40.760642,-111.902646,40.760641,-111.904098,0 days 21:50:08,0 days 21:50:08,0.130726,0.162459,-0.0026
2,5167502_to_depot,to_depot_1155977,"RoadId(start=83664915, end=1218577019, key=0)",40.760652,-111.904195,40.760773,-111.905303,0 days 21:50:08,0 days 21:50:09,0.102550,0.127443,-0.0003
3,5167502_to_depot,to_depot_1155977,"RoadId(start=376056428, end=1226596568, key=0)",40.762489,-111.908147,40.762822,-111.908131,0 days 21:50:11,0 days 21:50:12,0.045015,0.083913,0.0062
4,5167502_to_depot,to_depot_1155977,"RoadId(start=1218569199, end=1218569215, key=0)",40.760713,-111.894056,40.760716,-111.896843,0 days 21:50:03,0 days 21:50:05,0.243892,0.303095,-0.0080
...,...,...,...,...,...,...,...,...,...,...,...,...
96017,depot_to_5231660,from_depot_1159883,"RoadId(start=2155588269, end=83664915, key=0)",40.760614,-111.905175,40.760641,-111.904098,0 days 15:20:47.244417818,0 days 15:20:47.244417818,0.101366,0.125972,0.0055
96018,depot_to_5231660,from_depot_1159883,"RoadId(start=3529416919, end=1218568997, key=0)",40.760544,-111.888073,40.760549,-111.885373,0 days 15:20:52.244417818,0 days 15:20:53.244417818,0.233976,0.290773,-0.0046
96019,depot_to_5231660,from_depot_1159883,"RoadId(start=3581112093, end=3581112095, key=0)",40.764314,-111.908150,40.764319,-111.906862,0 days 15:20:44.244417818,0 days 15:20:44.244417818,0.108476,0.202212,0.0046
96020,depot_to_5231660,from_depot_1159883,"RoadId(start=3581112094, end=83650180, key=0)",40.764234,-111.905457,40.762824,-111.905471,0 days 15:20:45.244417818,0 days 15:20:45.244417818,0.165762,0.206000,-0.0026


In [509]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import multiprocessing as mp

from gtfsblocks import Feed, filter_blocks_by_route, add_deadhead, plot_trips_and_terminals

In [510]:
req_cols = {
        "stop_times": [
            "arrival_time",
            "departure_time",
            "shape_dist_traveled",
            "stop_id",
        ],
        "shapes": ["shape_dist_traveled"],
    }

In [511]:
feed = Feed.from_dir('/Users/yhe/github_repo/routee-transit/sample-inputs/saltlake/gtfs', columns=req_cols)

In [512]:
feed.stop_times

,departure_time,shape_dist_traveled,stop_id,arrival_time,trip_id,stop_sequence
0,0 days 05:45:00,NaN,5502,0 days 05:45:00,5182559,1
1,0 days 05:46:05,0.3150,13095,0 days 05:46:05,5182559,2
2,0 days 05:47:00,0.5740,5504,0 days 05:47:00,5182559,3
3,0 days 05:47:31,0.7910,13192,0 days 05:47:31,5182559,4
4,0 days 05:48:43,1.2973,21929,0 days 05:48:43,5182559,5
...,...,...,...,...,...,...
602361,0 days 23:31:15,3.2294,15669,0 days 23:31:15,5206456,4
602362,0 days 23:35:00,5.0390,20381,0 days 23:35:00,5206456,5
602363,0 days 23:37:46,6.2776,19178,0 days 23:37:46,5206456,6
602364,0 days 23:40:30,7.4995,24429,0 days 23:40:30,5206456,7


In [513]:
feed.stops

,stop_id,stop_lat,stop_lon
0,391,40.663345,-111.987053
1,506,40.660280,-111.987056
2,870,40.692636,-111.958136
3,887,40.693910,-111.957789
4,1022,40.772535,-111.837594
...,...,...,...
5229,25407,40.718417,-111.983917
5230,25410,40.580596,-111.829459
5231,25421,41.094656,-112.013359
5232,25423,40.983302,-111.911566


In [514]:
feed.shapes

,shape_pt_sequence,shape_pt_lat,shape_pt_lon,shape_dist_traveled,shape_id
0,1,40.76074,-111.88822,0.0000,226288
1,2,40.76140,-111.88823,0.0730,226288
2,3,40.76283,-111.88823,0.2340,226288
3,4,40.76363,-111.88824,0.3230,226288
4,5,40.76490,-111.88822,0.4650,226288
...,...,...,...,...,...
89585,122,40.76057,-111.86746,3.8002,228050
89586,123,40.76057,-111.86682,3.8542,228050
89587,124,40.76057,-111.86653,3.8782,228050
89588,125,40.76057,-111.86614,3.9112,228050


In [515]:
import pandas as pd
def create_depot_deadhead_trips(feed: any) -> pd.DataFrame:
    """Creat deadhead trips from and to depots for each block.
    Parameters
    ----------
    feed : Any
        GTFS feed object (e.g. result from read_in_gtfs).

    Returns
    -------
    pd.DataFrame: DataFrame with created deadhead trips.
    """

    existing_trips_df = feed.trips
    block_ids = existing_trips_df['block_id'].dropna().unique().tolist()
    # For each block id, create two deadhead trips: one from depot to first stop,
    # and one from last stop to depot.
    to_depot_trips = pd.DataFrame({'trip_id': [], 'route_id': [], 'service_id': [], 'block_id': [], 
                                   'shape_id': [], 'route_short_name': [], 'route_type': [], 'route_desc': [],
                                   'agency_id': []})
    from_depot_trips = pd.DataFrame({'trip_id': [], 'route_id': [], 'service_id': [], 'block_id': [], 
                                     'shape_id': [], 'route_short_name': [], 'route_type': [], 'route_desc': [],
                                     'agency_id': []})
    for block_id in block_ids:
        block_trips = existing_trips_df[existing_trips_df['block_id'] == block_id]
        if block_trips.empty:
            continue
        first_trip = block_trips.iloc[0]
        last_trip = block_trips.iloc[-1]
        # Create trip from depot to first stop
        from_depot_trip_id = f"depot_to_{first_trip['trip_id']}"
        from_depot_trip = {
            'trip_id': from_depot_trip_id,
            'route_id': first_trip['route_id'],
            'service_id': first_trip['service_id'],
            'block_id': block_id,
            'shape_id': f"from_depot_{block_id}",
            'route_short_name': first_trip.get('route_short_name', ''),
            'route_type': first_trip.get('route_type', 3),  # Default to bus
            'route_desc': f"Deadhead from depot to {first_trip['trip_id']}",
            'agency_id': first_trip.get('agency_id', None)
        }
        from_depot_trips = pd.concat([from_depot_trips, pd.DataFrame([from_depot_trip])], ignore_index=True)
        # Create trip from last stop to depot
        to_depot_trip_id = f"{last_trip['trip_id']}_to_depot"
        to_depot_trip = {
            'trip_id': to_depot_trip_id,
            'route_id': last_trip['route_id'],
            'service_id': last_trip['service_id'],
            'block_id': block_id,
            'shape_id': f"to_depot_{block_id}",
            'route_short_name': last_trip.get('route_short_name', ''),
            'route_type': last_trip.get('route_type', 3),  # Default to bus
            'route_desc': f"Deadhead from {last_trip['trip_id']} to depot",
            'agency_id': last_trip.get('agency_id', None)
        }
        to_depot_trips = pd.concat([to_depot_trips, pd.DataFrame([to_depot_trip])], ignore_index=True)

    deadhead_trips_df = pd.concat([from_depot_trips, to_depot_trips], ignore_index=True)
    return deadhead_trips_df

In [516]:
deadhead_trips_df = create_depot_deadhead_trips(feed)

In [517]:
deadhead_trips_df

,trip_id,route_id,service_id,block_id,shape_id,route_short_name,route_type,route_desc,agency_id
0,depot_to_5182559,19930,2,1155937,from_depot_1155937,,3.0,Deadhead from depot to 5182559,None
1,depot_to_5182560,19930,2,1155939,from_depot_1155939,,3.0,Deadhead from depot to 5182560,None
2,depot_to_5182561,19930,2,1155941,from_depot_1155941,,3.0,Deadhead from depot to 5182561,None
3,depot_to_5182562,19930,2,1155938,from_depot_1155938,,3.0,Deadhead from depot to 5182562,None
4,depot_to_5182563,19930,2,1155943,from_depot_1155943,,3.0,Deadhead from depot to 5182563,None
...,...,...,...,...,...,...,...,...,...
1923,5215687_to_depot,33219,22,1158500,to_depot_1158500,,3.0,Deadhead from 5215687 to depot,None
1924,5206437_to_depot,91962,22,1157711,to_depot_1157711,,3.0,Deadhead from 5206437 to depot,None
1925,5206438_to_depot,91962,22,1157710,to_depot_1157710,,3.0,Deadhead from 5206438 to depot,None
1926,5206455_to_depot,91962,22,1157709,to_depot_1157709,,3.0,Deadhead from 5206455 to depot,None


In [518]:
from typing import Any
from pathlib import Path
import os

import geopandas as gpd
from shapely.geometry import Point


def add_depot_to_blocks(feed: Any, path_to_depots: str | Path) -> tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]:
    """Add origin/destination depot geometry to each block id.

    Parameters
    ----------
    feed : Any
        GTFS feed object (e.g. result from read_in_gtfs).
    path_to_depots : str | Path
        Path to a vector file (GeoJSON/Shapefile) containing depot point geometries.

    Returns
    -------
    tuple[GeoDataFrame, GeoDataFrame]
        (first_stops_gdf, last_stops_gdf). Each GeoDataFrame contains the stop
        geometry (column 'stop_geometry') and the matched depot geometry
        (column 'depot_geometry').
    """

    # Process trips and stops dataframes in feed to get first and last stops of each block id
    trips_df = feed.trips
    stop_times_df = feed.stop_times
    stops_df = feed.stops   
    blocks_trips_stops = stop_times_df.merge(trips_df[['trip_id', 'block_id']], on='trip_id', how='left')
    blocks_trips_stops = blocks_trips_stops.merge(stops_df, on='stop_id', how='left')

    blocks_trips_stops = blocks_trips_stops.sort_values(by=['block_id', 'arrival_time'])
    first_stops = blocks_trips_stops.groupby('block_id').first().reset_index()
    last_stops = blocks_trips_stops.groupby('block_id').last().reset_index()

    first_stops = first_stops[['block_id', 'arrival_time', 'stop_lat', 'stop_lon']]
    last_stops = last_stops[['block_id', 'arrival_time', 'stop_lat', 'stop_lon']]

    first_stops['geometry'] = first_stops.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
    last_stops['geometry'] = last_stops.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
    first_stops_gdf = gpd.GeoDataFrame(first_stops, geometry='geometry', crs='EPSG:4326')
    last_stops_gdf = gpd.GeoDataFrame(last_stops, geometry='geometry', crs='EPSG:4326')

    # Read depot locations; ensure file exists
    if not os.path.exists(path_to_depots):
        raise FileNotFoundError(f"Depot file not found: {path_to_depots}")
    depots_df = gpd.read_file(path_to_depots)
    # Ensure depot geometries are points and in WGS84
    if depots_df.crs is None:
        depots_df = depots_df.set_crs(epsg=4326)
    else:
        depots_df = depots_df.to_crs(epsg=4326)

    # Create a simple mapping from depot index to geometry for fast lookup
    depots_geom_map = depots_df['geometry'].to_dict()

    # Project to Web Mercator (EPSG:3857) for distance computations
    proj_crs = "EPSG:3857"
    first_proj = first_stops_gdf.to_crs(proj_crs).reset_index(drop=True)
    last_proj = last_stops_gdf.to_crs(proj_crs).reset_index(drop=True)
    depots_proj = depots_df.to_crs(proj_crs).copy()

    # Use spatial join nearest to find the closest depot for each left row.
    # sjoin_nearest returns rows indexed by the left GeoDataFrame's index; if
    # multiple matches exist for the same left index we keep the closest (by
    # depot_dist_m). We then align back to the original stop GeoDataFrames by
    # resetting their indices to RangeIndex.
    first_nn = first_proj.sjoin_nearest(depots_proj[['geometry']], how='left', distance_col='depot_dist_m')
    last_nn = last_proj.sjoin_nearest(depots_proj[['geometry']], how='left', distance_col='depot_dist_m')

    # Drop duplicates 
    first_nn = first_nn.drop_duplicates(subset = ['block_id'])
    last_nn = last_nn.drop_duplicates(subset = ['block_id'])

    # Align and assign nearest depot index to the original stop GeoDataFrames
    first_stops_gdf = first_stops_gdf.copy().reset_index(drop=True)
    last_stops_gdf = last_stops_gdf.copy().reset_index(drop=True)
    first_stops_gdf['nearest_depot_idx'] = first_nn['index_right'].reindex(first_stops_gdf.index).astype('Int64')
    last_stops_gdf['nearest_depot_idx'] = last_nn['index_right'].reindex(last_stops_gdf.index).astype('Int64')

    # Set stop geometry and map matched depot geometry from the original
    # depots DataFrame (index_right references that index)
    first_stops_gdf['geometry_destination'] = first_stops_gdf.geometry
    last_stops_gdf['geometry_origin'] = last_stops_gdf.geometry
    first_stops_gdf['geometry_origin'] = first_stops_gdf['nearest_depot_idx'].map(depots_geom_map)
    last_stops_gdf['geometry_destination'] = last_stops_gdf['nearest_depot_idx'].map(depots_geom_map)

    # Set the arrival time as departure time for deadhead trip to depot for the last_stop_gdf
    last_stops_gdf['departure_time'] = last_stops_gdf['arrival_time']
    # Drop the arrival_time column for the last_stop_gdf
    last_stops_gdf = last_stops_gdf.drop(columns=['arrival_time'])

    # Keep only relevant columns and set stop_geometry as the active geometry
    first_stops_gdf = first_stops_gdf.drop(columns=['geometry'])
    first_stops_gdf = gpd.GeoDataFrame(first_stops_gdf, geometry='geometry_destination', crs='EPSG:4326')

    last_stops_gdf = last_stops_gdf.drop(columns=['geometry'])
    last_stops_gdf = gpd.GeoDataFrame(last_stops_gdf, geometry='geometry_origin', crs='EPSG:4326')

    return first_stops_gdf, last_stops_gdf

In [519]:
first_stops_gdf, last_stops_gdf = add_depot_to_blocks(feed, path_to_depots='/Users/yhe/github_repo/routee-transit/FTA_Depot/Transit_Depot.shp')

In [520]:
first_stops_gdf

,block_id,arrival_time,stop_lat,stop_lon,nearest_depot_idx,geometry_destination,geometry_origin
0,1153838,0 days 06:24:00,40.659597,-111.895438,738,POINT (-111.89544 40.6596),POINT (-111.91328 40.692613)
1,1153853,0 days 06:22:00,40.772150,-111.933123,746,POINT (-111.93312 40.77215),POINT (-111.909858 40.765876)
2,1153861,0 days 05:51:00,40.772150,-111.933123,746,POINT (-111.93312 40.77215),POINT (-111.909858 40.765876)
3,1153862,0 days 05:21:00,40.772150,-111.933123,746,POINT (-111.93312 40.77215),POINT (-111.909858 40.765876)
4,1153866,0 days 07:30:00,40.761431,-111.888044,1784,POINT (-111.88804 40.76143),POINT (-111.909225 40.764022)
...,...,...,...,...,...,...,...
959,1159879,0 days 15:19:00,40.660163,-111.895228,738,POINT (-111.89523 40.66016),POINT (-111.91328 40.692613)
960,1159880,0 days 06:09:00,40.760491,-111.866141,1784,POINT (-111.86614 40.76049),POINT (-111.909225 40.764022)
961,1159881,0 days 15:09:00,40.760491,-111.866141,1784,POINT (-111.86614 40.76049),POINT (-111.909225 40.764022)
962,1159882,0 days 05:25:00,40.760491,-111.866141,1784,POINT (-111.86614 40.76049),POINT (-111.909225 40.764022)


In [521]:
last_stops_gdf

,block_id,stop_lat,stop_lon,nearest_depot_idx,geometry_origin,geometry_destination,departure_time
0,1153838,40.774009,-111.904983,746,POINT (-111.90498 40.77401),POINT (-111.909858 40.765876),0 days 19:11:00
1,1153853,40.763678,-111.909788,1784,POINT (-111.90979 40.76368),POINT (-111.909225 40.764022),0 days 20:13:00
2,1153861,40.693438,-111.960025,740,POINT (-111.96002 40.69344),POINT (-111.918302 40.692123),0 days 19:25:00
3,1153862,40.638454,-112.027011,740,POINT (-112.02701 40.63845),POINT (-111.918302 40.692123),0 days 18:57:00
4,1153866,40.760010,-111.888422,1784,POINT (-111.88842 40.76001),POINT (-111.909225 40.764022),0 days 11:36:00
...,...,...,...,...,...,...,...
959,1159879,40.660163,-111.895228,738,POINT (-111.89523 40.66016),POINT (-111.91328 40.692613),1 days 00:09:00
960,1159880,40.760491,-111.866141,1784,POINT (-111.86614 40.76049),POINT (-111.909225 40.764022),0 days 14:48:00
961,1159881,40.760491,-111.866141,1784,POINT (-111.86614 40.76049),POINT (-111.909225 40.764022),0 days 23:18:00
962,1159882,40.760491,-111.866141,1784,POINT (-111.86614 40.76049),POINT (-111.909225 40.764022),0 days 09:35:00


In [522]:
# last_stops_gdf['distance_m'] = last_stops_gdf.apply(
#     lambda row: geodesic(
#         (row.geometry_origin.y, row.geometry_origin.x),
#         (row.geometry_destination.y, row.geometry_destination.x)
#     ).meters,
#     axis=1)

In [523]:
# last_stops_gdf['travel_time_sec'] = (last_stops_gdf['distance_m'] / 30000) * 3600
# last_stops_gdf

In [524]:
# last_stops_gdf['arrival_time'] = last_stops_gdf['departure_time'] + pd.to_timedelta(last_stops_gdf['travel_time_sec'], unit='s')

In [525]:
len(last_stops_gdf['block_id'].unique())

964

In [526]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.distance import geodesic

def create_depot_deadhead_stops(first_stops_gdf: gpd.GeoDataFrame, 
                                    last_stops_gdf: gpd.GeoDataFrame,
                                    deadhead_trips: pd.DataFrame) -> pd.DataFrame:
    """Creat stop_times and stops for deadhead trips from and to depots for each block to generate the feed object for depot deadhead trips.
    Parameters
    ----------
    first_stops_gdf: gpd.GeoDataFrame
        GeoDataFrame of first stops for each block id with depot geometry (result from add_depot_to_blocks.py).
    last_stops_gdf: gpd.GeoDataFrame
        GeoDataFrame of last stops for each block id with depot geometry (result from add_depot_to_blocks.py).
    deadhead_trips: pd.DataFrame
        deadhead trip results from create_depot_deadhead_trips.py.
    Returns
    -------
    pd.DataFrame
        DataFrame of stop_times and stops for the deadhead trips.
    """

    from_depot = first_stops_gdf.copy()
    to_depot = last_stops_gdf.copy()

    # Calculate distance from depot to first stop
    from_depot['distance_m'] = from_depot.apply(
    lambda row: geodesic(
        (row.geometry_origin.y, row.geometry_origin.x),
        (row.geometry_destination.y, row.geometry_destination.x)
    ).meters,
    axis=1)
    # Calculate distance from last stop to depot
    to_depot['distance_m'] = to_depot.apply(
    lambda row: geodesic(
        (row.geometry_origin.y, row.geometry_origin.x),
        (row.geometry_destination.y, row.geometry_destination.x)
    ).meters,
    axis=1)
    # Assume average speed of 30 km/h (to be consistant with the number adopted in gtfs_feature_processing.py) 
    # to estimate travel time
    from_depot['travel_time_sec'] = (from_depot['distance_m'] / 30000) * 3600
    to_depot['travel_time_sec'] = (to_depot['distance_m'] / 30000) * 3600
    # Calculate departure time from depot for deadhead trip to first stop
    from_depot['departure_time'] = from_depot['arrival_time'] - pd.to_timedelta(from_depot['travel_time_sec'], unit='s')
    # Calculate arrival time at depot for deadhead trip from last stop
    to_depot['arrival_time'] = to_depot['departure_time'] + pd.to_timedelta(to_depot['travel_time_sec'], unit='s')

    # Create stop_times df for deadhead trips
    deadhead_trips_df = deadhead_trips.copy()
    stop_times_df = pd.DataFrame(columns=['trip_id', 'stop_sequence', 'arrival_time', 'stop_id', 'departure_time', 'shape_dist_traveled'])
    stop_times_df['trip_id'] =  deadhead_trips_df['trip_id'].repeat(2).values
    stop_times_df['stop_sequence'] = [1, 2] * len(deadhead_trips_df)
    stop_times_df['arrival_time'] = [x for pair in zip(from_depot['departure_time'].to_list(), from_depot['arrival_time'].to_list()) for x in pair] \
                                    + [x for pair in zip(to_depot['departure_time'].to_list(), to_depot['arrival_time'].to_list()) for x in pair]
    stop_times_df['stop_id'] = range(1, len(stop_times_df) + 1)
    stop_times_df['stop_id'] = stop_times_df['stop_id'].apply(lambda x: f"depot_deadhead_{x}")
    stop_times_df['departure_time'] = stop_times_df['arrival_time']
    stop_times_df['shape_dist_traveled'] = 0.0

    # Create stops df for deadhead trips
    stops_df = pd.DataFrame(columns=['stop_id', 'stop_lat', 'stop_lon'])
    stops_df['stop_id'] = stop_times_df['stop_id']

    x_start_from_depot = from_depot.geometry_origin.apply(lambda p: p.x).to_numpy()
    x_end_from_depot = from_depot.geometry_destination.apply(lambda p: p.x).to_numpy()
    x_start_to_depot = to_depot.geometry_origin.apply(lambda p: p.x).to_numpy()
    x_end_to_depot = to_depot.geometry_destination.apply(lambda p: p.x).to_numpy()
    stop_lon_from_depot = np.ravel(np.column_stack((x_start_from_depot, x_end_from_depot)))
    stop_lon_to_depot = np.ravel(np.column_stack((x_start_to_depot, x_end_to_depot)))
                        

    y_start_from_depot = from_depot.geometry_origin.apply(lambda p: p.y).to_numpy()
    y_end_from_depot = from_depot.geometry_destination.apply(lambda p: p.y).to_numpy()
    y_start_to_depot = to_depot.geometry_origin.apply(lambda p: p.y).to_numpy()
    y_end_to_depot = to_depot.geometry_destination.apply(lambda p: p.y).to_numpy()
    stop_lat_from_depot = np.ravel(np.column_stack((y_start_from_depot, y_end_from_depot)))
    stop_lat_to_depot = np.ravel(np.column_stack((y_start_to_depot, y_end_to_depot)))

    stops_df['stop_lat'] = list(stop_lat_from_depot) + list(stop_lat_to_depot)
    stops_df['stop_lon'] = list(stop_lon_from_depot) + list(stop_lon_to_depot)

    return stop_times_df, stops_df

In [527]:
deadhead_stop_times_df, deadhead_stops_df = create_depot_deadhead_stops(first_stops_gdf, last_stops_gdf, deadhead_trips_df)

In [528]:
deadhead_stop_times_df

,trip_id,stop_sequence,arrival_time,stop_id,departure_time,shape_dist_traveled
0,depot_to_5182559,1,0 days 06:16:04.255134041,depot_deadhead_1,0 days 06:16:04.255134041,0.0
1,depot_to_5182559,2,0 days 06:24:00,depot_deadhead_2,0 days 06:24:00,0.0
2,depot_to_5182560,1,0 days 06:17:49.904480890,depot_deadhead_3,0 days 06:17:49.904480890,0.0
3,depot_to_5182560,2,0 days 06:22:00,depot_deadhead_4,0 days 06:22:00,0.0
4,depot_to_5182561,1,0 days 05:46:49.904480890,depot_deadhead_5,0 days 05:46:49.904480890,0.0
...,...,...,...,...,...,...
3851,5206438_to_depot,2,0 days 23:25:19.073439322,depot_deadhead_3852,0 days 23:25:19.073439322,0.0
3852,5206455_to_depot,1,0 days 09:35:00,depot_deadhead_3853,0 days 09:35:00,0.0
3853,5206455_to_depot,2,0 days 09:42:19.073439322,depot_deadhead_3854,0 days 09:42:19.073439322,0.0
3854,5206456_to_depot,1,0 days 19:35:00,depot_deadhead_3855,0 days 19:35:00,0.0


In [529]:
deadhead_stops_df

,stop_id,stop_lat,stop_lon
0,depot_deadhead_1,40.692613,-111.913280
1,depot_deadhead_2,40.659597,-111.895438
2,depot_deadhead_3,40.765876,-111.909858
3,depot_deadhead_4,40.772150,-111.933123
4,depot_deadhead_5,40.765876,-111.909858
...,...,...,...
3851,depot_deadhead_3852,40.764022,-111.909225
3852,depot_deadhead_3853,40.760491,-111.866141
3853,depot_deadhead_3854,40.764022,-111.909225
3854,depot_deadhead_3855,40.760491,-111.866141


In [530]:
import pandas as pd
import geopandas as gpd
# single-global-graph approach; per-row cached graphs removed
import multiprocessing as mp
from typing import Tuple, Any

# optional imports for routing/map-matching kept at module level so we don't import per-row
try:
    import osmnx as ox_module
    import networkx as nx_module
except Exception:
    ox_module = None
    nx_module = None

# Optionally hold a single pre-fetched graph for the whole study area.
GLOBAL_GRAPH = None


# Note: per-row cached graph fetch removed. This module now requires a single
# pre-fetched study-area graph assigned to GLOBAL_GRAPH (via bbox in
# `add_deadhead_trips`) and all routing workers use that graph.


def _parallel_map(func, iterable, n_processes: int | None, chunksize: int = 8):
    """Simple wrapper to run func over iterable using multiprocessing when requested."""
    if not n_processes or n_processes <= 1:
        return [func(x) for x in iterable]
    results = []
    with mp.Pool(n_processes) as pool:
        for item in pool.imap(func, iterable, chunksize=chunksize):
            results.append(item)
    return results

def _process_deadhead_trip_row(args: Tuple[Any, ...]):
    """Worker that computes shortest-path and returns per-point shape rows.

    args: (idx, start_x, start_y, end_x, end_y, block_id, network_type, road_buffer_m)

    Returns a list of dicts with fields:
      - 'shape_id' (block id)
      - 'shape_pt_sequence' (1-based int)
      - 'shape_pt_lon'
      - 'shape_pt_lat'
      - 'shape_dist_traveled' (cumulative km from route start)
    """
    from shapely.geometry import LineString
    from shapely.ops import linemerge, unary_union
    import math

    def _haversine_km(lat1, lon1, lat2, lon2):
        R = 6371.0
        phi1 = math.radians(lat1)
        phi2 = math.radians(lat2)
        dphi = math.radians(lat2 - lat1)
        dlambda = math.radians(lon2 - lon1)
        a = math.sin(dphi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2.0) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return R * c

    (
        idx,
        start_x,
        start_y,
        end_x,
        end_y,
        block_id,
        network_type,
        road_buffer_m,
    ) = args

    if ox_module is None or nx_module is None:
        return []

    G = GLOBAL_GRAPH
    if G is None:
        return []

    try:
        src = ox_module.nearest_nodes(G, start_x, start_y)
        dst = ox_module.nearest_nodes(G, end_x, end_y)
        route_nodes = nx_module.shortest_path(G, src, dst, weight='length')

        edge_geoms = []
        for u, v in zip(route_nodes[:-1], route_nodes[1:]):
            data = G.get_edge_data(u, v)
            if not data:
                continue
            key = list(data.keys())[0]
            attr = data[key]
            geom = attr.get('geometry')
            if geom is None:
                ux = G.nodes[u].get('x')
                uy = G.nodes[u].get('y')
                vx = G.nodes[v].get('x')
                vy = G.nodes[v].get('y')
                geom = LineString([(ux, uy), (vx, vy)])
            edge_geoms.append(geom)

        if not edge_geoms: # Return a link from start to end if no route found
            return []
            # return [{
            #     'shape_id': block_id,
            #     'shape_pt_sequence': 1,
            #     'shape_pt_lon': float(start_x),
            #     'shape_pt_lat': float(start_y),
            #     'shape_dist_traveled': 0.0,
            # }, {
            #     'shape_id': block_id,
            #     'shape_pt_sequence': 2,
            #     'shape_pt_lon': float(end_x),
            #     'shape_pt_lat': float(end_y),
            #     'shape_dist_traveled': _haversine_km(start_y, start_x, end_y, end_x),
            # }]

        try:
            merged = linemerge(edge_geoms)
        except Exception:
            merged = unary_union(edge_geoms)
        if getattr(merged, 'geom_type', '') == 'MultiLineString':
            coords = []
            for part in merged:
                coords.extend(list(part.coords))
            merged = LineString(coords)

        coords = list(merged.coords)  # list of (lon, lat)
        rows = []
        prev_lat = None
        prev_lon = None
        cum_km = 0.0
        for seq, (lon, lat) in enumerate(coords, start=1):
            if prev_lat is not None:
                seg_km = _haversine_km(prev_lat, prev_lon, lat, lon)
                cum_km += seg_km
            else:
                cum_km = 0.0
            rows.append({
                'shape_id': block_id,
                'shape_pt_sequence': int(seq),
                'shape_pt_lon': float(lon),
                'shape_pt_lat': float(lat),
                'shape_dist_traveled': float(cum_km),
            })
            prev_lat, prev_lon = lat, lon

        return rows
    except Exception:
        return []


def add_deadhead_trips(
    df: gpd.GeoDataFrame,
    o_col: str = 'geometry_origin',
    d_col: str = 'geometry_destination',
    network_type: str = 'drive',
    road_buffer_m: int = 2000,
    n_processes: int | None = None,
    bbox: tuple[float, float, float, float] | None = None,
    ) -> pd.DataFrame:
    """Compute deadhead route shapes between origin and destination.

    For each row in `df` this computes a shortest-path on the OSM network between the
    origin (o_col) and destination (d_col) using a single pre-fetched study-area graph
    (fetched from `bbox`). Returns a pandas DataFrame with per-point GTFS-like shape
    rows: ['shape_id', 'shape_pt_sequence', 'shape_pt_lon', 'shape_pt_lat', 'shape_dist_traveled']
    where `shape_dist_traveled` is cumulative distance in kilometers from the route start.
    """
    # Require bbox: this function uses a single study-area graph for all routing.
    global GLOBAL_GRAPH
    if bbox is None:
        raise ValueError('bbox is required: provide a study-area bounding box (minx, miny, maxx, maxy)')
    if ox_module is None:
        raise ImportError('osmnx is required to fetch a global bbox graph')
    # bbox expected as (minx, miny, maxx, maxy)
    try:
        minx, miny, maxx, maxy = bbox
    except Exception:
        raise ValueError('bbox must be a 4-tuple (minx, miny, maxx, maxy)')
    # ox.graph_from_bbox signature: (north, south, east, west)
    GLOBAL_GRAPH = ox_module.graph_from_bbox(bbox, network_type='drive')
    
    task_args = []
    for idx, r in df.iterrows():
        origin = r.get(o_col)
        destination = r.get(d_col)
        if origin is None or destination is None:
            continue
        task_args.append((
            idx,
            float(origin.x),
            float(origin.y),
            float(destination.x),
            float(destination.y),
            r.get('block_id') if 'block_id' in r else None,
            network_type,
            road_buffer_m,
        ))

    results = _parallel_map(_process_deadhead_trip_row, task_args, n_processes)

    # results is a list of lists (per-route rows). Flatten and build DataFrame
    shape_rows = []
    for route_rows in results:
        if not route_rows:
            continue
        # route_rows is a list of dicts from the worker
        shape_rows.extend(route_rows)

    out_df = pd.DataFrame(
        shape_rows,
        columns=['shape_id', 'shape_pt_sequence', 'shape_pt_lon', 'shape_pt_lat', 'shape_dist_traveled'],
    )
    return out_df

In [531]:
all_points = pd.concat([first_stops_gdf['geometry_origin'], first_stops_gdf['geometry_destination']])
lons = all_points.apply(lambda p: p.x)
lats = all_points.apply(lambda p: p.y)
min_lon, max_lon = lons.min(), lons.max() # Bounding box
min_lat, max_lat = lats.min(), lats.max() # Bounding box
buffer_deg_lat = 0.018     # Roughly 2 km buffer in degrees
buffer_deg_lon = 0.022     # Roughly 2 km buffer in degrees
miny = min_lat - buffer_deg_lat
maxy = max_lat + buffer_deg_lat
minx = min_lon - buffer_deg_lon
maxx = max_lon + buffer_deg_lon
from_depot_deadhead_shapes_df = add_deadhead_trips(
    df = first_stops_gdf,
    n_processes = 1,
    bbox = [minx, miny, maxx, maxy]
    )
from_depot_deadhead_shapes_df['shape_id'] = from_depot_deadhead_shapes_df['shape_id'].apply(lambda x: 'from_depot_' + x)

In [532]:
from_depot_deadhead_shapes_df

,shape_id,shape_pt_sequence,shape_pt_lon,shape_pt_lat,shape_dist_traveled
0,from_depot_1153838,1,-111.911194,40.691679,0.000000
1,from_depot_1153838,2,-111.911192,40.691096,0.064860
2,from_depot_1153838,3,-111.911192,40.690962,0.079671
3,from_depot_1153838,4,-111.911193,40.690444,0.137282
4,from_depot_1153838,5,-111.911193,40.690361,0.146522
...,...,...,...,...,...
215106,from_depot_1159883,168,-111.865329,40.760759,4.043595
215107,from_depot_1159883,169,-111.865524,40.760760,4.060010
215108,from_depot_1159883,170,-111.866019,40.760758,4.101735
215109,from_depot_1159883,171,-111.866120,40.760758,4.110199


In [533]:
from_depot_counts = from_depot_deadhead_shapes_df.groupby('shape_id').size().reset_index(name='num_rows')

In [535]:
from_depot_counts[from_depot_counts['num_rows'] == from_depot_counts['num_rows'].min()]

,shape_id,num_rows
427,from_depot_1154669,16
615,from_depot_1156071,16
628,from_depot_1156084,16
635,from_depot_1156091,16
643,from_depot_1156099,16
648,from_depot_1156104,16
907,from_depot_1157814,16


In [536]:
# # Check blocks with only two points and the distance

# df_group = from_depot_deadhead_shapes_df.groupby('shape_id').agg({'shape_pt_sequence':'last',
#                                                                'shape_dist_traveled':'last'}).reset_index()

In [537]:
# df_group[df_group.shape_pt_sequence == 2]

In [538]:
# block_ids = from_depot_deadhead_shapes_df['shape_id'].apply(lambda x: x.split('_')[2]).to_list()

In [539]:
len(from_depot_deadhead_shapes_df['shape_id'].unique())

918

In [540]:
# all_block_ids = first_stops_gdf['block_id'].unique().tolist()

In [541]:
# [x for x in all_block_ids if x not in block_ids]

In [542]:
all_points = pd.concat([last_stops_gdf['geometry_origin'], last_stops_gdf['geometry_destination']])
lons = all_points.apply(lambda p: p.x)
lats = all_points.apply(lambda p: p.y)
min_lon, max_lon = lons.min(), lons.max() # Bounding box
min_lat, max_lat = lats.min(), lats.max() # Bounding box
buffer_deg_lat = 0.018     # Roughly 2 km buffer in degrees
buffer_deg_lon = 0.022     # Roughly 2 km buffer in degrees         
miny = min_lat - buffer_deg_lat
maxy = max_lat + buffer_deg_lat
minx = min_lon - buffer_deg_lon
maxx = max_lon + buffer_deg_lon
to_depot_deadhead_shapes_df = add_deadhead_trips(
    df = last_stops_gdf,
    n_processes = 1,
    bbox = [minx, miny, maxx, maxy]
    )
to_depot_deadhead_shapes_df['shape_id'] = to_depot_deadhead_shapes_df['shape_id'].apply(lambda x: 'to_depot_' + x)

In [543]:
to_depot_deadhead_shapes_df

,shape_id,shape_pt_sequence,shape_pt_lon,shape_pt_lat,shape_dist_traveled
0,to_depot_1153838,1,-111.904832,40.773694,0.000000
1,to_depot_1153838,2,-111.904827,40.773579,0.012795
2,to_depot_1153838,3,-111.904824,40.772821,0.097081
3,to_depot_1153838,4,-111.904822,40.772783,0.101253
4,to_depot_1153838,5,-111.904817,40.772751,0.104874
...,...,...,...,...,...
220317,to_depot_1159883,146,-111.908131,40.762822,3.733470
220318,to_depot_1159883,147,-111.908130,40.762974,3.750372
220319,to_depot_1159883,148,-111.908129,40.763096,3.763937
220320,to_depot_1159883,149,-111.908146,40.763888,3.851992


In [544]:
len(to_depot_deadhead_shapes_df['shape_id'].unique())

916

In [545]:
to_depot_counts = to_depot_deadhead_shapes_df.groupby('shape_id').size().reset_index(name='num_rows')

In [546]:
to_depot_counts[to_depot_counts['num_rows'] == to_depot_counts['num_rows'].min()]

,shape_id,num_rows
245,to_depot_1154320,7
248,to_depot_1154323,7
249,to_depot_1154324,7
254,to_depot_1154329,7
255,to_depot_1154330,7
260,to_depot_1154335,7
261,to_depot_1154336,7
819,to_depot_1157694,7


In [553]:
to_depot_deadhead_shapes_df[to_depot_deadhead_shapes_df['shape_id']=='to_depot_1157694']

,shape_id,shape_pt_sequence,shape_pt_lon,shape_pt_lat,shape_dist_traveled
194640,to_depot_1157694,1,-111.908264,40.762828,0.000000
194641,to_depot_1157694,2,-111.908251,40.762827,0.001141
194642,to_depot_1157694,3,-111.908131,40.762822,0.011258
194643,to_depot_1157694,4,-111.908130,40.762974,0.028160
194644,to_depot_1157694,5,-111.908129,40.763096,0.041726
194645,to_depot_1157694,6,-111.908146,40.763888,0.129781
194646,to_depot_1157694,7,-111.908150,40.764314,0.177073


In [446]:
deadhead_shapes_df = pd.concat([from_depot_deadhead_shapes_df, to_depot_deadhead_shapes_df], ignore_index=True)

In [447]:
deadhead_shapes_df

,shape_id,shape_pt_sequence,shape_pt_lon,shape_pt_lat,shape_dist_traveled
0,from_depot_1153838,1,-111.911194,40.691679,0.000000
1,from_depot_1153838,2,-111.911192,40.691096,0.064860
2,from_depot_1153838,3,-111.911192,40.690962,0.079671
3,from_depot_1153838,4,-111.911193,40.690444,0.137282
4,from_depot_1153838,5,-111.911193,40.690361,0.146522
...,...,...,...,...,...
285,to_depot_1153838,76,-111.908483,40.765084,1.619989
286,to_depot_1153838,77,-111.908927,40.765057,1.657529
287,to_depot_1153838,78,-111.909406,40.765003,1.698267
288,to_depot_1153838,79,-111.909596,40.764999,1.714299


In [554]:
deadhead_shapes_df = to_depot_deadhead_shapes_df[to_depot_deadhead_shapes_df['shape_id']=='to_depot_1154320']

In [555]:
len(deadhead_shapes_df['shape_id'].unique())

1

In [556]:
deadhead_trips_df = deadhead_trips_df[deadhead_trips_df['shape_id'].isin(deadhead_shapes_df['shape_id'].unique())]

In [557]:
deadhead_trips_df

,trip_id,route_id,service_id,block_id,shape_id,route_short_name,route_type,route_desc,agency_id
1102,5173635_to_depot,27613,2,1154320,to_depot_1154320,,3.0,Deadhead from 5173635 to depot,None


In [451]:
trips_df = feed.trips
trips_df = trips_df[trips_df['trip_id']=='5171596']
trips_df

,trip_id,route_id,service_id,block_id,shape_id
869,5171596,27610,2,1153838,226474


In [452]:
trips_df = pd.concat([trips_df, deadhead_trips_df], ignore_index=True)

In [558]:
trips_df = deadhead_trips_df

In [454]:
shapes_df = feed.shapes
shapes_df = shapes_df[shapes_df['shape_id']== '226474']
shapes_df

,shape_pt_sequence,shape_pt_lat,shape_pt_lon,shape_dist_traveled,shape_id
18342,1,40.774836,-111.904882,0.0000,226474
18343,2,40.774066,-111.904876,0.0860,226474
18344,3,40.773807,-111.904843,0.1152,226474
18345,4,40.773727,-111.904776,0.1260,226474
18346,5,40.773686,-111.904690,0.1346,226474
...,...,...,...,...,...
18641,300,40.660389,-111.894530,15.7904,226474
18642,301,40.660440,-111.894630,15.8006,226474
18643,302,40.660470,-111.894769,15.8124,226474
18644,303,40.660410,-111.894890,15.8246,226474


In [455]:
shapes_df = pd.concat([shapes_df, deadhead_shapes_df], ignore_index=True)

In [456]:
shapes_df

,shape_pt_sequence,shape_pt_lat,shape_pt_lon,shape_dist_traveled,shape_id
0,1,40.774836,-111.904882,0.000000,226474
1,2,40.774066,-111.904876,0.086000,226474
2,3,40.773807,-111.904843,0.115200,226474
3,4,40.773727,-111.904776,0.126000,226474
4,5,40.773686,-111.904690,0.134600,226474
...,...,...,...,...,...
589,76,40.765084,-111.908483,1.619989,to_depot_1153838
590,77,40.765057,-111.908927,1.657529,to_depot_1153838
591,78,40.765003,-111.909406,1.698267,to_depot_1153838
592,79,40.764999,-111.909596,1.714299,to_depot_1153838


In [559]:
shapes_df = deadhead_shapes_df
shapes_df

,shape_id,shape_pt_sequence,shape_pt_lon,shape_pt_lat,shape_dist_traveled
60301,to_depot_1154320,1,-111.908264,40.762828,0.000000
60302,to_depot_1154320,2,-111.908251,40.762827,0.001141
60303,to_depot_1154320,3,-111.908131,40.762822,0.011258
60304,to_depot_1154320,4,-111.908130,40.762974,0.028160
60305,to_depot_1154320,5,-111.908129,40.763096,0.041726
60306,to_depot_1154320,6,-111.908146,40.763888,0.129781
60307,to_depot_1154320,7,-111.908150,40.764314,0.177073


In [560]:
feed.trips = pd.concat([feed.trips, deadhead_trips_df], ignore_index=True)

In [561]:
feed.trips

,trip_id,route_id,service_id,block_id,shape_id,route_short_name,route_type,route_desc,agency_id
0,5182559,19930,2,1155937,226296,NaN,NaN,NaN,NaN
1,5182560,19930,2,1155939,226296,NaN,NaN,NaN,NaN
2,5182561,19930,2,1155941,226296,NaN,NaN,NaN,NaN
3,5182562,19930,2,1155938,226296,NaN,NaN,NaN,NaN
4,5182563,19930,2,1155943,226296,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12033,5206453,91962,22,1157709,227290,NaN,NaN,NaN,NaN
12034,5206454,91962,22,1157708,227290,NaN,NaN,NaN,NaN
12035,5206455,91962,22,1157709,227290,NaN,NaN,NaN,NaN
12036,5206456,91962,22,1157708,227290,NaN,NaN,NaN,NaN


In [562]:
feed.shapes = pd.concat([feed.shapes, deadhead_shapes_df], ignore_index=True)

In [563]:
feed.shapes

,shape_pt_sequence,shape_pt_lat,shape_pt_lon,shape_dist_traveled,shape_id
0,1,40.760740,-111.888220,0.000000,226288
1,2,40.761400,-111.888230,0.073000,226288
2,3,40.762830,-111.888230,0.234000,226288
3,4,40.763630,-111.888240,0.323000,226288
4,5,40.764900,-111.888220,0.465000,226288
...,...,...,...,...,...
89592,3,40.762822,-111.908131,0.011258,to_depot_1154320
89593,4,40.762974,-111.908130,0.028160,to_depot_1154320
89594,5,40.763096,-111.908129,0.041726,to_depot_1154320
89595,6,40.763888,-111.908146,0.129781,to_depot_1154320


In [564]:
feed.stop_times = pd.concat([feed.stop_times, deadhead_stop_times_df], ignore_index=True)

In [565]:
feed.stop_times

,departure_time,shape_dist_traveled,stop_id,arrival_time,trip_id,stop_sequence
0,0 days 05:45:00,NaN,5502,0 days 05:45:00,5182559,1
1,0 days 05:46:05,0.3150,13095,0 days 05:46:05,5182559,2
2,0 days 05:47:00,0.5740,5504,0 days 05:47:00,5182559,3
3,0 days 05:47:31,0.7910,13192,0 days 05:47:31,5182559,4
4,0 days 05:48:43,1.2973,21929,0 days 05:48:43,5182559,5
...,...,...,...,...,...,...
606217,0 days 23:25:19.073439322,0.0000,depot_deadhead_3852,0 days 23:25:19.073439322,5206438_to_depot,2
606218,0 days 09:35:00,0.0000,depot_deadhead_3853,0 days 09:35:00,5206455_to_depot,1
606219,0 days 09:42:19.073439322,0.0000,depot_deadhead_3854,0 days 09:42:19.073439322,5206455_to_depot,2
606220,0 days 19:35:00,0.0000,depot_deadhead_3855,0 days 19:35:00,5206456_to_depot,1


In [566]:
feed.stops = pd.concat([feed.stops, deadhead_stops_df], ignore_index=True)

In [567]:
feed.stops

,stop_id,stop_lat,stop_lon
0,391,40.663345,-111.987053
1,506,40.660280,-111.987056
2,870,40.692636,-111.958136
3,887,40.693910,-111.957789
4,1022,40.772535,-111.837594
...,...,...,...
9085,depot_deadhead_3852,40.764022,-111.909225
9086,depot_deadhead_3853,40.760491,-111.866141
9087,depot_deadhead_3854,40.764022,-111.909225
9088,depot_deadhead_3855,40.760491,-111.866141


In [568]:
def upsample_shape(shape_df: pd.DataFrame) -> pd.DataFrame:
    """Upsample a GTFS shape DataFrame to generate a roughly 1 Hz GPS trace.

    Interpolates latitude, longitude, and distance traveled, assuming a constant speed.
    The function performs the following steps:

    * Calculates the distance between consecutive shape points using great-circle distance
    * Computes the cumulative distance traveled along the shape
    * Assigns timestamps based on constant speed (30 km/h)
    * Resamples and interpolates the shape to 1-second intervals
    * Returns DataFrame with interpolated coordinates, timestamps, and distances

    Args:
        shape_df: DataFrame containing GTFS shape points with columns
            'shape_pt_lat', 'shape_pt_lon', and 'shape_id'.

    Returns:
        Upsampled DataFrame with columns 'shape_pt_lat', 'shape_pt_lon',
        'shape_dist_traveled', 'timestamp', and 'shape_id', sampled at 1 Hz.
    """

    # Shift latitude and longitude to get previous point
    shape_df["prev_latitude"] = shape_df["shape_pt_lat"].shift()
    shape_df["prev_longitude"] = shape_df["shape_pt_lon"].shift()

    # Calculate the distance between consecutive points using great_circle
    # TODO: move away from apply() for speed
    shape_df["distance_km"] = shape_df.apply(
        lambda row: great_circle(
            (row["prev_latitude"], row["prev_longitude"]),  # Previous point
            (row["shape_pt_lat"], row["shape_pt_lon"]),  # Current point
        ).kilometers
        if pd.notnull(row["prev_latitude"])
        else 0,
        axis=1,
    )

    # Calculate total distance
    total_distance_km = shape_df["distance_km"].sum()

    # Use calculated total distance instead of shape_dist_traveled
    shape_df["shape_dist_traveled"] = shape_df["distance_km"].cumsum()

    # Speed is assumed to be 30 km/h, which is about 10 (8.33) m per second/node
    shape_df["segment_duration_delta"] = (
        shape_df["shape_dist_traveled"]
        / shape_df["shape_dist_traveled"].max()
        * datetime.timedelta(seconds=round(total_distance_km / 30 * 3600))
    )
    shape_df["segment_duration_delta"] = shape_df["segment_duration_delta"].apply(
        lambda x: datetime.timedelta(seconds=round(x.total_seconds()))
    )
    # Define an arbitrary date to convert from timedelta to datetime
    date_tmp = datetime.datetime(2023, 9, 3)
    shape_df["timestamp"] = (
        datetime.timedelta(seconds=0) + shape_df["segment_duration_delta"] + date_tmp
    )

    # Upsample to 1s
    shape_id_tmp = shape_df.shape_id.iloc[0]
    shape_df = (
        shape_df[["shape_pt_lat", "shape_pt_lon", "timestamp", "shape_dist_traveled"]]
        .drop_duplicates(subset=["timestamp"])
        .set_index("timestamp")
        .resample("1s")
        .interpolate(method="linear")
    )

    # Now we have the 1 Hz gps trace for each trip with timestamp
    shape_df = shape_df.reset_index(drop=True)
    shape_df["shape_id"] = shape_id_tmp

    return shape_df

In [569]:
df_shape_list = [group for _, group in shapes_df.groupby("shape_id")]
upsampled_shapes_list = []
for i in df_shape_list:
    upsampled_shapes_list += [upsample_shape(i)]
# n_processes=1
# with mp.Pool(n_processes) as pool:
#     upsampled_shapes_list = pool.map(upsample_shape, df_shape_list)
# logger.debug("Original shapes length: {}".format(len(shapes_df)))
# logger.debug(
#     "Upsampled shapes length: {}".format(len(pd.concat(upsampled_shapes_list)))
# )

In [570]:
def match_shape_to_osm(upsampled_shape_df: pd.DataFrame) -> pd.DataFrame:
    """Match a given GTFS shape DataFrame to the OpenStreetMap (OSM) road network.

    This function uses mappymatch to add OSM network information to the shape trace.
    The trace should be upsampled beforehand to approximately 1 Hz/8 m for the most
    accurate expected mapping performance. The function creates a Trace from the input
    DataFrame, constructs a geofence around the trace, extracts the OSM road network
    within the geofence, and applies the mappymatch LCSS matcher to align the trace to
    the network. The output DataFrame retains the full shape while adding network
    information to each row.

    Args:
        upsampled_shape_df (pd.DataFrame): DataFrame containing the shape points with
            latitude and longitude columns ("shape_pt_lat" and "shape_pt_lon").
    Returns:
        pd.DataFrame: A DataFrame combining the original upsampled shape points with
            their corresponding OSM network matches.
    """
    # Create mappymatch trace
    trace = Trace.from_dataframe(
        upsampled_shape_df, lat_column="shape_pt_lat", lon_column="shape_pt_lon"
    )
    # Create geofence and use it to pull network
    geofence = Geofence.from_trace(trace, padding=1e3)
    nxmap = NxMap.from_geofence(geofence, network_type=NetworkType.DRIVE)
    # Run map matching algorithm
    matcher = LCSSMatcher(nxmap)
    matches = matcher.match_trace(trace).matches_to_dataframe()
    # Combine shape with network details
    df_result = pd.concat([upsampled_shape_df, matches], axis=1)
    return df_result

In [571]:
len(upsampled_shapes_list)

1

In [572]:
upsampled_shapes_list[-1]

,shape_pt_lat,shape_pt_lon,shape_dist_traveled,shape_id
0,40.762828,-111.908264,0.000000,to_depot_1154320
1,40.762822,-111.908131,0.011258,to_depot_1154320
2,40.762898,-111.908130,0.019709,to_depot_1154320
3,40.762974,-111.908130,0.028160,to_depot_1154320
4,40.763035,-111.908130,0.034943,to_depot_1154320
5,40.763096,-111.908129,0.041726,to_depot_1154320
6,40.763176,-111.908131,0.050531,to_depot_1154320
7,40.763255,-111.908133,0.059337,to_depot_1154320
8,40.763334,-111.908134,0.068142,to_depot_1154320
9,40.763413,-111.908136,0.076948,to_depot_1154320


In [573]:
matched_shapes_list = []
for j in upsampled_shapes_list:
    matched_shapes_list += [match_shape_to_osm(j)]

# # Run mapmatching in parallel for each shape
# with mp.Pool(n_processes) as pool:
#     matched_shapes_list = pool.map(match_shape_to_osm, upsampled_shapes_list)

# logger.info("Finished map matching")

In [574]:
matched_shapes_df = pd.concat(matched_shapes_list)

In [575]:
def extend_trip_traces(
    trips_df: pd.DataFrame,
    matched_shapes_df: pd.DataFrame,
    feed: Feed,
    add_stop_flag: bool = False,
    n_processes: int | None = mp.cpu_count(),
) -> pd.DataFrame:
    """Extend trip shapes with stop details and estimated timestamps from GTFS.

    This function processes GTFS trip and shape data to:

    * Summarize stop times for each trip (first/last stop and times)
    * Merge stop time summaries into the trips DataFrame
    * Attach stop coordinates to stop times
    * Merge trip and shape data to create ordered trip traces
    * Optionally, attach stop indicators to shape trace points
    * Estimate timestamps for each trace point based on scheduled trip duration and distance

    Args:
        trips_df: DataFrame containing trip information, including
            'trip_id' and 'shape_id'.
        matched_shapes_df: DataFrame with shape points matched to trips,
            including 'shape_id' and 'shape_dist_traveled'.
        feed: GTFS feed object containing 'stop_times' and 'stops'
            DataFrames.
        add_stop_flag: If True, attaches stop indicators to shape trace
            points. Defaults to False.
        n_processes: Number of processes to run in parallel using
            multiprocessing. Defaults to mp.cpu_count().

    Returns:
        A list of DataFrames, one per trip, with extended trace information
        including estimated timestamps.
    """
    # Start by summarizing stop times: get first and last stop, plus start/end times
    stop_times_by_trip = (
        feed.stop_times.groupby("trip_id")
        .agg(
            {
                "arrival_time": "first",
                "departure_time": "last",
                "stop_id": ["first", "last"],
            }
        )
        .reset_index()
    )
    stop_times_by_trip.columns = [
        "trip_id",
        "o_time",
        "d_time",
        "o_stop_id",
        "d_stop_id",
    ]

    # Add start/end times and stops to trips DF
    # TODO: consider doing this with gtfsblocks add_trip_data()
    trips_df = pd.merge(trips_df, stop_times_by_trip, how="left", on="trip_id")
    trips_df["o_time"] = pd.to_timedelta(trips_df["o_time"])
    trips_df["d_time"] = pd.to_timedelta(trips_df["d_time"])
    trips_df["trip_duration"] = trips_df["d_time"] - trips_df["o_time"]

    # Add stop coordinates to stop_times
    stop_times_ext = feed.stop_times[["trip_id", "stop_sequence", "stop_id"]].merge(
        feed.stops[["stop_id", "stop_lat", "stop_lon"]], on="stop_id"
    )

    # calculate approximate timestamps for each GPS trace
    # TODO: I think this big merge can be avoided
    trip_shape = pd.merge(
        trips_df[["trip_id", "shape_id", "o_time", "d_time"]],
        matched_shapes_df,
        how="left",
        on="shape_id",
    )
    trip_shape = trip_shape.sort_values(
        by=["trip_id", "shape_dist_traveled"]
    ).reset_index(drop=True)
    trip_shapes_list = [item for _, item in trip_shape.groupby("trip_id")]

    # Attach stops to shape traces. Note that this just adds a dummy variable column
    # indicating whether or not a stop is located at a given point on the shape.
    if add_stop_flag:
        attach_stop_partial = partial(
            add_stop_flags_to_shape, stop_times_ext=stop_times_ext
        )
        with mp.Pool(n_processes) as pool:
            trip_shapes_list = pool.map(attach_stop_partial, trip_shapes_list)

    # Attach timestamps to each trip. These are simply based on the scheduled trip
    # duration and shape_dist_traveled, assuming a constant speed for the entire trip.
    # TODO: improve timestamp estimates
    with mp.Pool(n_processes) as pool:
        trips_with_timestamps_list = pool.map(
            estimate_trip_timestamps, trip_shapes_list
        )
    logger.info("Finished attaching timestamps")
    return pd.concat(trips_with_timestamps_list)

In [576]:
def estimate_trip_timestamps(trip_shape_df: pd.DataFrame) -> pd.DataFrame:
    """Estimate timestamps for each shape point of a trip based on distance traveled.

    Args:
        trip_shape_df (pd.DataFrame): DataFrame containing trip shape data with columns:
            - 'shape_dist_traveled': Cumulative distance traveled along the shape.
            - 'o_time': Origin time (datetime) of the trip.
            - 'd_time': Destination time (datetime) of the trip.
    Returns:
        pd.DataFrame: Modified DataFrame with additional columns:
            - 'segment_duration_delta': Estimated duration for each segment as timedelta.
            - 'timestamp': Estimated timestamp for each segment.
            - 'Datetime_nearest5': Timestamp rounded to the nearest 5 minutes.
            - 'hour': Hour component of the rounded timestamp.
            - 'minute': Minute component of the rounded timestamp.
    """
    trip_shape_df["segment_duration_delta"] = (
        trip_shape_df["shape_dist_traveled"]
        / (trip_shape_df["shape_dist_traveled"].max()+0.0001)
        * (trip_shape_df["d_time"] - trip_shape_df["o_time"])
    )
    trip_shape_df["segment_duration_delta"] = trip_shape_df[
        "segment_duration_delta"
    ].apply(lambda x: datetime.timedelta(seconds=round(x.total_seconds())))
    trip_shape_df["timestamp"] = (
        trip_shape_df["o_time"] + trip_shape_df["segment_duration_delta"]
    )

    ## get hour and minute of gps timestamp
    trip_shape_df["Datetime_nearest5"] = trip_shape_df["timestamp"].dt.round("5min")
    trip_shape_df["hour"] = trip_shape_df["Datetime_nearest5"].dt.components["hours"]
    trip_shape_df["minute"] = trip_shape_df["Datetime_nearest5"].dt.components[
        "minutes"
    ]

    return trip_shape_df

In [577]:
stop_times_by_trip = (
        feed.stop_times.groupby("trip_id")
        .agg(
            {
                "arrival_time": "first",
                "departure_time": "last",
                "stop_id": ["first", "last"],
            }
        )
        .reset_index()
    )
stop_times_by_trip.columns = [
        "trip_id",
        "o_time",
        "d_time",
        "o_stop_id",
        "d_stop_id",
    ]

In [578]:
trips_df = pd.merge(trips_df, stop_times_by_trip, how="left", on="trip_id")
trips_df["o_time"] = pd.to_timedelta(trips_df["o_time"])
trips_df["d_time"] = pd.to_timedelta(trips_df["d_time"])
trips_df["trip_duration"] = trips_df["d_time"] - trips_df["o_time"]

# Add stop coordinates to stop_times
stop_times_ext = feed.stop_times[["trip_id", "stop_sequence", "stop_id"]].merge(
    feed.stops[["stop_id", "stop_lat", "stop_lon"]], on="stop_id"
)

# calculate approximate timestamps for each GPS trace
# TODO: I think this big merge can be avoided
trip_shape = pd.merge(
    trips_df[["trip_id", "shape_id", "o_time", "d_time"]],
    matched_shapes_df,
    how="left",
    on="shape_id",
)
trip_shape = trip_shape.sort_values(
    by=["trip_id", "shape_dist_traveled"]
).reset_index(drop=True)
trip_shapes_list = [item for _, item in trip_shape.groupby("trip_id")]

In [579]:
trips_with_timestamps_list = []
for m in trip_shapes_list:
    trips_with_timestamps_list += [estimate_trip_timestamps(m)]

In [580]:
trips_df_ext = pd.concat(trips_with_timestamps_list)

In [581]:
trip_links_df = (
        trips_df_ext.groupby(by=["trip_id", "shape_id", "road_id"])
        .agg(
            start_lat=pd.NamedAgg("shape_pt_lat", "first"),
            start_lon=pd.NamedAgg("shape_pt_lon", "first"),
            end_lat=pd.NamedAgg("shape_pt_lat", "last"),
            end_lon=pd.NamedAgg("shape_pt_lon", "last"),
            geom=pd.NamedAgg("geom", "first"),
            start_timestamp=pd.NamedAgg("timestamp", "first"),
            end_timestamp=pd.NamedAgg("timestamp", "last"),
            kilometers=pd.NamedAgg("kilometers", "mean"),
            travel_time_minutes=pd.NamedAgg("travel_time", "mean"),
        )
        .reset_index()
    )
trip_links_df["travel_time_minutes"] /= 60
trips_df_list = [t_df for _, t_df in trip_links_df.groupby("trip_id")]

In [582]:
def _convert_to_coord_df(
    trip_link_df: pd.DataFrame,
) -> pd.DataFrame:
    """Convert a trip DataFrame to a coordinate DataFrame for gradeit processing.

    Args:
        trip_link_df (pd.DataFrame): Trip DataFrame where geometry has been aggregated
            by link.

    Returns:
        pd.DataFrame: DataFrame with 'lat' and 'lon' columns for each point in the trip.
    """
    trip_coords_df = trip_link_df[["start_lat", "start_lon"]].rename(
        columns={"start_lat": "lat", "start_lon": "lon"}
    )
    last_rw = trip_link_df.iloc[-1][["end_lat", "end_lon"]].T
    trip_coords_df = pd.concat(
        [
            trip_coords_df,
            last_rw.to_frame().rename({"end_lat": "lat", "end_lon": "lon"}).T,
        ],
        ignore_index=True,
    )
    return trip_coords_df

In [583]:
from nrel.routee.transit.prediction.grade.tile_resolution import TileResolution
tile_resolution = TileResolution.ONE_THIRD_ARC_SECOND
if isinstance(tile_resolution, str):
        tile_resolution = TileResolution.from_string(tile_resolution)

trip_coords_list = [
    _convert_to_coord_df(trip_link_df) for trip_link_df in trips_df_list
]

In [584]:
from nrel.routee.transit.prediction.grade.download import CACHE_DIR, download_usgs_tiles

all_points = [
    (row["lat"], row["lon"])
    for trip_coords_df in trip_coords_list
    for _, row in trip_coords_df.iterrows()
]

download_usgs_tiles(
    lat_lon_pairs=all_points,
    resolution=tile_resolution,
    output_dir=CACHE_DIR,
)

INFO:nrel.routee.transit.prediction.grade.download:Downloading 1 USGS tiles at ONE_THIRD_ARC_SECOND resolution.
2025-10-08 14:38:46,728 [INFO] - Downloading 1 USGS tiles at ONE_THIRD_ARC_SECOND resolution.
INFO:nrel.routee.transit.prediction.grade.download:cache/n41w112/USGS_13_n41w112.tif already exists, skipping
2025-10-08 14:38:46,734 [INFO] - cache/n41w112/USGS_13_n41w112.tif already exists, skipping


[PosixPath('cache/n41w112/USGS_13_n41w112.tif')]

In [585]:
def add_grade_to_trip(
    trip_link_df: pd.DataFrame,
    tile_resolution: TileResolution = TileResolution.ONE_THIRD_ARC_SECOND,
) -> pd.DataFrame:
    """Use gradeit to add grade and elevation columns to a trip DataFrame.

    Args:
        trip_link_df (pd.DataFrame): Trip DataFrame where geometry has been aggregated
            by link.
        tile_resolution (TileResolution): The resolution of the USGS tiles to use for
            elevation and grade calculations. Defaults to ONE_ARC_SECOND.

    Returns:
        pd.DataFrame: Trip DataFrame with elevation and grade columns added.
    """
    if isinstance(tile_resolution, str):
        tile_resolution = TileResolution.from_string(tile_resolution)

    # Reset the index to make sure things go as expected when we combine DFs later
    trip_link_df = trip_link_df.reset_index(drop=True)

    # Input DF is link level. Extract the coordinates of all points by grabbing the
    # start coords of each link, then appending the end coords of the last one.
    trip_coords_df = _convert_to_coord_df(trip_link_df)

    # Run gradeit on the coordinate-level DF
    gradeit_out = gradeit(
        df=trip_coords_df,
        source="usgs-local",
        usgs_db_path=CACHE_DIR,
        lat_col="lat",
        lon_col="lon",
        filtering=True,
    ).rename(columns={"grade_dec_unfiltered": "grade"})

    # When calculating grades, the first point from gradeit has zero distance/grade.
    # We'll shift these to appropriately match to links.
    gradeit_cols = ["grade"]  # for now, only include a single grade column
    trip_link_df.loc[:, gradeit_cols] = gradeit_out.shift(-1)[:-1].loc[:, gradeit_cols]

    return trip_link_df

In [593]:
def safe_add_grade_to_trip(
    trip_link_df: pd.DataFrame,
    tile_resolution="ONE_THIRD_ARC_SECOND",
    min_points: int = 5
) -> pd.DataFrame:
    """
    Wrapper around add_grade_to_trip to handle short trips safely.
    Assigns grade = 0 for trips with too few coordinates.
    """
    if len(trip_link_df) < min_points:
        trip_link_df = trip_link_df.copy()
        trip_link_df["grade"] = 0.0
        return trip_link_df

    try:
        return add_grade_to_trip(trip_link_df, tile_resolution=tile_resolution)
    except ValueError as e:
        if "polyorder must be less than window_length" in str(e):
            trip_link_df = trip_link_df.copy()
            trip_link_df["grade"] = 0.0
            return trip_link_df
        else:
            raise e

In [594]:
gradeit_partial = partial(safe_add_grade_to_trip, tile_resolution=tile_resolution)

In [595]:
from gradeit.gradeit import gradeit
trips_with_grade = gradeit_partial(trips_df_list[0])

In [596]:
trips_with_grade

,trip_id,shape_id,road_id,start_lat,start_lon,end_lat,end_lon,geom,start_timestamp,end_timestamp,kilometers,travel_time_minutes,grade
0,5173635_to_depot,to_depot_1154320,"(1226596332, 1226596568, 0)",40.762828,-111.908264,40.762822,-111.908131,LINESTRING (-12457571.008567024 4977421.552919...,0 days 19:08:00,0 days 19:08:08,0.011258,0.018852,0.0
1,5173635_to_depot,to_depot_1154320,"(1226596568, 3581112093, 0)",40.762898,-111.908130,40.764314,-111.908150,LINESTRING (-12457556.158546953 4977420.715178...,0 days 19:08:13,0 days 19:09:59,0.165815,0.309100,0.0


In [3]:
import datetime
import logging
import multiprocessing as mp
from functools import partial
from pathlib import Path
from typing import Union

import geopandas as gpd
import pandas as pd
from geopy.distance import great_circle
from gtfsblocks import Feed, filter_blocks_by_route
from mappymatch.constructs.geofence import Geofence
from mappymatch.constructs.trace import Trace
from mappymatch.maps.nx.nx_map import NetworkType, NxMap
from mappymatch.matchers.lcss.lcss import LCSSMatcher

from nrel.routee.transit.prediction.grade.add_grade import run_gradeit_parallel
from nrel.routee.transit.prediction.grade.tile_resolution import TileResolution

logger = logging.getLogger("gtfs_feature_processing")

KM_TO_METERS = 1000
FT_TO_METERS = 0.3048
FT_TO_MILES = 0.000189394

def read_in_gtfs(
    path_to_feed: str | Path,
    date_incl: str | datetime.date | None = None,
    routes_incl: list[str] | None = None,
) -> tuple[pd.DataFrame, pd.DataFrame, Feed]:
    """
    Reads a GTFS feed from a directory, optionally filtering trips by date and route.
    Args:
        path_to_feed (str | Path): Path to the GTFS feed directory.
        date_incl (str | datetime.date | None, optional): Date to filter trips.
            If None, includes all dates.
        routes_incl (list[str] | None, optional): List of route_short_name values to
            filter trips based on. If None, includes all routes.
    Returns:
        tuple:
            - trips_df (pd.DataFrame): DataFrame of filtered trips.
            - shapes_df (pd.DataFrame): DataFrame of shapes associated with the
                filtered trips.
            - feed (Feed): The loaded GTFS feed object.
    Raises:
        ValueError: If no trips are found for the specified date or routes.
    """

    req_cols = {
        "stop_times": [
            "arrival_time",
            "departure_time",
            "shape_dist_traveled",
            "stop_id",
        ],
        "shapes": ["shape_dist_traveled"],
    }
    feed = Feed.from_dir(path_to_feed, columns=req_cols)
    agencies_incl = feed.agency.agency_name.unique().tolist()

    logger.info(
        f"Feed includes trips for the following agencies: {agencies_incl}. "
        f"There are {len(feed.trips)} trips and "
        f"{feed.shapes.shape_id.nunique()} shapes"
    )

    # 1.5) Filter down feed by date and route
    if date_incl is not None:
        trips_df = feed.get_trips_from_date(date_incl)
        if len(trips_df) == 0:
            raise ValueError(f"Feed does not contain any trips on {date_incl}")
    else:
        trips_df = feed.get_trips_from_sids(feed.trips.service_id.unique().tolist())

    if routes_incl is not None:
        trips_df = filter_blocks_by_route(
            trips=trips_df,
            routes=routes_incl,
            route_column="route_short_name",
            route_method="exclusive",
        )

        if len(trips_df) == 0:
            raise ValueError(
                "There are no active trips on your selected routes and date."
            )

    shapes_incl = trips_df.shape_id.unique()
    shapes_df = feed.shapes[feed.shapes.shape_id.isin(shapes_incl)]
    logger.info(
        f"Restricted feed to {len(trips_df)} trips and {len(shapes_incl)} shapes"
    )
    # TODO: establish a routee-transit input class and return an object instead
    return trips_df, shapes_df, feed

input_directory = '/Users/yhe/github_repo/routee-transit/sample-inputs/saltlake/gtfs'
date_incl = None
routes_incl = None
# 1) Process GTFS inputs
trips_df, shapes_df, feed = read_in_gtfs(
    path_to_feed=input_directory,
    date_incl=date_incl,
    routes_incl=routes_incl,
)

INFO:gtfs_feature_processing:Feed includes trips for the following agencies: ['Utah Transit Authority']. There are 12037 trips and 238 shapes
2025-10-05 08:27:11,544 [INFO] - Feed includes trips for the following agencies: ['Utah Transit Authority']. There are 12037 trips and 238 shapes
INFO:gtfs_feature_processing:Restricted feed to 12037 trips and 199 shapes
2025-10-05 08:27:11,551 [INFO] - Restricted feed to 12037 trips and 199 shapes


In [4]:
trips_df

,trip_id,route_id,service_id,block_id,shape_id,route_short_name,route_type,route_desc,agency_id
0,5182559,19930,2,1155937,226296,470,3,NaN,NaN
1,5182560,19930,2,1155939,226296,470,3,NaN,NaN
2,5182561,19930,2,1155941,226296,470,3,NaN,NaN
3,5182562,19930,2,1155938,226296,470,3,NaN,NaN
4,5182563,19930,2,1155943,226296,470,3,NaN,NaN
...,...,...,...,...,...,...,...,...,...
14902,5206452,91962,22,1157708,227290,BB2,3,NaN,NaN
14903,5206453,91962,22,1157709,227290,BB2,3,NaN,NaN
14904,5206454,91962,22,1157708,227290,BB2,3,NaN,NaN
14905,5206455,91962,22,1157709,227290,BB2,3,NaN,NaN


In [24]:
feed = gtfs_slc

In [46]:
type(feed)

gtfsblocks.gtfs.Feed

In [47]:
feed.shapes = pd.concat([feed.shapes,feed.shapes])

In [50]:
feed.shapes

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,226288,40.76074,-111.88822,1
1,226288,40.76140,-111.88823,2
2,226288,40.76283,-111.88823,3
3,226288,40.76363,-111.88824,4
4,226288,40.76490,-111.88822,5
...,...,...,...,...
89585,228050,40.76057,-111.86746,122
89586,228050,40.76057,-111.86682,123
89587,228050,40.76057,-111.86653,124
89588,228050,40.76057,-111.86614,125


In [48]:
shapes = feed.shapes
shapes

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,226288,40.76074,-111.88822,1
1,226288,40.76140,-111.88823,2
2,226288,40.76283,-111.88823,3
3,226288,40.76363,-111.88824,4
4,226288,40.76490,-111.88822,5
...,...,...,...,...
89585,228050,40.76057,-111.86746,122
89586,228050,40.76057,-111.86682,123
89587,228050,40.76057,-111.86653,124
89588,228050,40.76057,-111.86614,125


In [45]:
stop_times = feed.stop_times
stop_times

,trip_id,stop_sequence,arrival_time,stop_id
0,5182559,1,0 days 05:45:00,5502
1,5182559,2,0 days 05:46:05,13095
2,5182559,3,0 days 05:47:00,5504
3,5182559,4,0 days 05:47:31,13192
4,5182559,5,0 days 05:48:43,21929
...,...,...,...,...
602361,5206456,4,0 days 23:31:15,15669
602362,5206456,5,0 days 23:35:00,20381
602363,5206456,6,0 days 23:37:46,19178
602364,5206456,7,0 days 23:40:30,24429


In [51]:
feed.stops

,stop_id,stop_lat,stop_lon
0,391,40.663345,-111.987053
1,506,40.660280,-111.987056
2,870,40.692636,-111.958136
3,887,40.693910,-111.957789
4,1022,40.772535,-111.837594
...,...,...,...
5229,25407,40.718417,-111.983917
5230,25410,40.580596,-111.829459
5231,25421,41.094656,-112.013359
5232,25423,40.983302,-111.911566


In [36]:
def upsample_shape(shape_df: pd.DataFrame) -> pd.DataFrame:
    """Upsample a GTFS shape DataFrame to generate a roughly 1 Hz GPS trace.

    Interpolates latitude, longitude, and distance traveled, assuming a constant speed.
    The function performs the following steps:

    * Calculates the distance between consecutive shape points using great-circle distance
    * Computes the cumulative distance traveled along the shape
    * Assigns timestamps based on constant speed (30 km/h)
    * Resamples and interpolates the shape to 1-second intervals
    * Returns DataFrame with interpolated coordinates, timestamps, and distances

    Args:
        shape_df: DataFrame containing GTFS shape points with columns
            'shape_pt_lat', 'shape_pt_lon', and 'shape_id'.

    Returns:
        Upsampled DataFrame with columns 'shape_pt_lat', 'shape_pt_lon',
        'shape_dist_traveled', 'timestamp', and 'shape_id', sampled at 1 Hz.
    """

    # Shift latitude and longitude to get previous point
    shape_df["prev_latitude"] = shape_df["shape_pt_lat"].shift()
    shape_df["prev_longitude"] = shape_df["shape_pt_lon"].shift()

    # Calculate the distance between consecutive points using great_circle
    # TODO: move away from apply() for speed
    shape_df["distance_km"] = shape_df.apply(
        lambda row: great_circle(
            (row["prev_latitude"], row["prev_longitude"]),  # Previous point
            (row["shape_pt_lat"], row["shape_pt_lon"]),  # Current point
        ).kilometers
        if pd.notnull(row["prev_latitude"])
        else 0,
        axis=1,
    )

    # Calculate total distance
    total_distance_km = shape_df["distance_km"].sum()

    # Use calculated total distance instead of shape_dist_traveled
    shape_df["shape_dist_traveled"] = shape_df["distance_km"].cumsum()

    # Speed is assumed to be 30 km/h, which is about 10 (8.33) m per second/node
    shape_df["segment_duration_delta"] = (
        shape_df["shape_dist_traveled"]
        / shape_df["shape_dist_traveled"].max()
        * datetime.timedelta(seconds=round(total_distance_km / 30 * 3600))
    )
    shape_df["segment_duration_delta"] = shape_df["segment_duration_delta"].apply(
        lambda x: datetime.timedelta(seconds=round(x.total_seconds()))
    )
    # Define an arbitrary date to convert from timedelta to datetime
    date_tmp = datetime.datetime(2023, 9, 3)
    shape_df["timestamp"] = (
        datetime.timedelta(seconds=0) + shape_df["segment_duration_delta"] + date_tmp
    )

    # Upsample to 1s
    shape_id_tmp = shape_df.shape_id.iloc[0]
    shape_df = (
        shape_df[["shape_pt_lat", "shape_pt_lon", "timestamp", "shape_dist_traveled"]]
        .drop_duplicates(subset=["timestamp"])
        .set_index("timestamp")
        .resample("1s")
        .interpolate(method="linear")
    )

    # Now we have the 1 Hz gps trace for each trip with timestamp
    shape_df = shape_df.reset_index(drop=True)
    shape_df["shape_id"] = shape_id_tmp

    return shape_df

In [38]:
upsampled_shape = upsample_shape(shapes[shapes['shape_id']=='226288'])

/var/folders/31/y4mwl1qs14z8r1x7g524813cc7rklk/T/ipykernel_28054/1873780147.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shape_df["prev_latitude"] = shape_df["shape_pt_lat"].shift()
/var/folders/31/y4mwl1qs14z8r1x7g524813cc7rklk/T/ipykernel_28054/1873780147.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shape_df["prev_longitude"] = shape_df["shape_pt_lon"].shift()
/var/folders/31/y4mwl1qs14z8r1x7g524813cc7rklk/T/ipykernel_28054/1873780147.py:28: SettingWithCopyWarning: 
A value is trying to be

In [40]:
shapes[shapes['shape_id']=='226288']

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,226288,40.76074,-111.88822,1
1,226288,40.76140,-111.88823,2
2,226288,40.76283,-111.88823,3
3,226288,40.76363,-111.88824,4
4,226288,40.76490,-111.88822,5
...,...,...,...,...
938,226288,41.23380,-111.97858,939
939,226288,41.23418,-111.97857,940
940,226288,41.23496,-111.97856,941
941,226288,41.23681,-111.97852,942


In [39]:
upsampled_shape

,shape_pt_lat,shape_pt_lon,shape_dist_traveled,shape_id
0,40.760740,-111.888220,0.000000,226288
1,40.760813,-111.888221,0.008155,226288
2,40.760887,-111.888222,0.016310,226288
3,40.760960,-111.888223,0.024465,226288
4,40.761033,-111.888224,0.032619,226288
...,...,...,...,...
8500,41.236816,-111.979226,70.830076,226288
8501,41.236817,-111.979327,70.838514,226288
8502,41.236818,-111.979428,70.846952,226288
8503,41.236819,-111.979529,70.855391,226288


In [ ]:
def match_shape_to_osm(upsampled_shape_df: pd.DataFrame) -> pd.DataFrame:
    """Match a given GTFS shape DataFrame to the OpenStreetMap (OSM) road network.

    This function uses mappymatch to add OSM network information to the shape trace.
    The trace should be upsampled beforehand to approximately 1 Hz/8 m for the most
    accurate expected mapping performance. The function creates a Trace from the input
    DataFrame, constructs a geofence around the trace, extracts the OSM road network
    within the geofence, and applies the mappymatch LCSS matcher to align the trace to
    the network. The output DataFrame retains the full shape while adding network
    information to each row.

    Args:
        upsampled_shape_df (pd.DataFrame): DataFrame containing the shape points with
            latitude and longitude columns ("shape_pt_lat" and "shape_pt_lon").
    Returns:
        pd.DataFrame: A DataFrame combining the original upsampled shape points with
            their corresponding OSM network matches.
    """
    # Create mappymatch trace
    trace = Trace.from_dataframe(
        upsampled_shape_df, lat_column="shape_pt_lat", lon_column="shape_pt_lon"
    )
    # Create geofence and use it to pull network
    geofence = Geofence.from_trace(trace, padding=1e3)
    nxmap = NxMap.from_geofence(geofence, network_type=NetworkType.DRIVE)
    # Run map matching algorithm
    matcher = LCSSMatcher(nxmap)
    matches = matcher.match_trace(trace).matches_to_dataframe()
    # Combine shape with network details
    df_result = pd.concat([upsampled_shape_df, matches], axis=1)
    return df_result

In [43]:
osm_shape = match_shape_to_osm(upsampled_shape)

In [44]:
osm_shape

,shape_pt_lat,shape_pt_lon,shape_dist_traveled,shape_id,coordinate_id,distance_to_road,road_id,geom,origin_junction_id,destination_junction_id,road_key,kilometers,travel_time
0,40.760740,-111.888220,0.000000,226288,0,7.142993,"(3529416922, 3529416926, 0)",LINESTRING (-12455332.551718356 4977115.179528...,3529416922,3529416926,0,0.223323,16.651984
1,40.760813,-111.888221,0.008155,226288,1,7.391765,"(3529416922, 3529416926, 0)",LINESTRING (-12455332.551718356 4977115.179528...,3529416922,3529416926,0,0.223323,16.651984
2,40.760887,-111.888222,0.016310,226288,2,7.648223,"(3529416922, 3529416926, 0)",LINESTRING (-12455332.551718356 4977115.179528...,3529416922,3529416926,0,0.223323,16.651984
3,40.760960,-111.888223,0.024465,226288,3,7.883910,"(3529416922, 3529416926, 0)",LINESTRING (-12455332.551718356 4977115.179528...,3529416922,3529416926,0,0.223323,16.651984
4,40.761033,-111.888224,0.032619,226288,4,8.119598,"(3529416922, 3529416926, 0)",LINESTRING (-12455332.551718356 4977115.179528...,3529416922,3529416926,0,0.223323,16.651984
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8500,41.236816,-111.979226,70.830076,226288,8500,80.523130,"(84416040, 84411887, 0)",LINESTRING (-12465395.666706832 5047059.901024...,84416040,84411887,0,0.207346,11.595547
8501,41.236817,-111.979327,70.838514,226288,8501,91.756662,"(84416040, 84411887, 0)",LINESTRING (-12465395.666706832 5047059.901024...,84416040,84411887,0,0.207346,11.595547
8502,41.236818,-111.979428,70.846952,226288,8502,102.990193,"(84416040, 84411887, 0)",LINESTRING (-12465395.666706832 5047059.901024...,84416040,84411887,0,0.207346,11.595547
8503,41.236819,-111.979529,70.855391,226288,8503,114.223724,"(84416040, 84411887, 0)",LINESTRING (-12465395.666706832 5047059.901024...,84416040,84411887,0,0.207346,11.595547


In [26]:
from typing import Any
from pathlib import Path
import os

import geopandas as gpd
from shapely.geometry import Point


def add_depot_to_blocks(feed: Any, path_to_depots: str | Path) -> tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]:
    """Add origin/destination depot geometry to each block id.

    Parameters
    ----------
    feed : Any
        GTFS feed object (e.g. result from read_in_gtfs).
    path_to_depots : str | Path
        Path to a vector file (GeoJSON/Shapefile) containing depot point geometries.

    Returns
    -------
    tuple[GeoDataFrame, GeoDataFrame]
        (first_stops_gdf, last_stops_gdf). Each GeoDataFrame contains the stop
        geometry (column 'stop_geometry') and the matched depot geometry
        (column 'depot_geometry').
    """

    # Process trips and stops dataframes in feed to get first and last stops of each block id
    trips_df = feed.trips
    stop_times_df = feed.stop_times
    stops_df = feed.stops   
    blocks_trips_stops = stop_times_df.merge(trips_df[['trip_id', 'block_id']], on='trip_id', how='left')
    blocks_trips_stops = blocks_trips_stops.merge(stops_df, on='stop_id', how='left')

    blocks_trips_stops = blocks_trips_stops.sort_values(by=['block_id', 'arrival_time'])
    first_stops = blocks_trips_stops.groupby('block_id').first().reset_index()
    last_stops = blocks_trips_stops.groupby('block_id').last().reset_index()

    first_stops = first_stops[['block_id', 'arrival_time', 'stop_lat', 'stop_lon']]
    last_stops = last_stops[['block_id', 'arrival_time', 'stop_lat', 'stop_lon']]

    first_stops['geometry'] = first_stops.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
    last_stops['geometry'] = last_stops.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
    first_stops_gdf = gpd.GeoDataFrame(first_stops, geometry='geometry', crs='EPSG:4326')
    last_stops_gdf = gpd.GeoDataFrame(last_stops, geometry='geometry', crs='EPSG:4326')

    # Read in depot locations
    # Read depot locations; ensure file exists
    if not os.path.exists(path_to_depots):
        raise FileNotFoundError(f"Depot file not found: {path_to_depots}")
    depots_df = gpd.read_file(path_to_depots)
    # Ensure depot geometries are points and in WGS84
    if depots_df.crs is None:
        depots_df = depots_df.set_crs(epsg=4326)
    else:
        depots_df = depots_df.to_crs(epsg=4326)

    # Create a simple mapping from depot index to geometry for fast lookup
    depots_geom_map = depots_df['geometry'].to_dict()

    # Project to Web Mercator (EPSG:3857) for distance computations
    proj_crs = "EPSG:3857"
    first_proj = first_stops_gdf.to_crs(proj_crs).reset_index(drop=True)
    last_proj = last_stops_gdf.to_crs(proj_crs).reset_index(drop=True)
    depots_proj = depots_df.to_crs(proj_crs).copy()

    # Use spatial join nearest to find the closest depot for each left row.
    # sjoin_nearest returns rows indexed by the left GeoDataFrame's index; if
    # multiple matches exist for the same left index we keep the closest (by
    # depot_dist_m). We then align back to the original stop GeoDataFrames by
    # resetting their indices to RangeIndex.
    first_nn = first_proj.sjoin_nearest(depots_proj[['geometry']], how='left', distance_col='depot_dist_m')
    last_nn = last_proj.sjoin_nearest(depots_proj[['geometry']], how='left', distance_col='depot_dist_m')

    # Drop duplicates 
    first_nn = first_nn.drop_duplicates(subset = ['block_id'])
    last_nn = last_nn.drop_duplicates(subset = ['block_id'])

    # Align and assign nearest depot index to the original stop GeoDataFrames
    first_stops_gdf = first_stops_gdf.copy().reset_index(drop=True)
    last_stops_gdf = last_stops_gdf.copy().reset_index(drop=True)
    first_stops_gdf['nearest_depot_idx'] = first_nn['index_right'].reindex(first_stops_gdf.index).astype('Int64')
    last_stops_gdf['nearest_depot_idx'] = last_nn['index_right'].reindex(last_stops_gdf.index).astype('Int64')

    # Set stop geometry and map matched depot geometry from the original
    # depots DataFrame (index_right references that index)
    first_stops_gdf['geometry_destination'] = first_stops_gdf.geometry
    last_stops_gdf['geometry_origin'] = last_stops_gdf.geometry
    first_stops_gdf['geometry_origin'] = first_stops_gdf['nearest_depot_idx'].map(depots_geom_map)
    last_stops_gdf['geometry_destination'] = last_stops_gdf['nearest_depot_idx'].map(depots_geom_map)

    # Keep only relevant columns and set stop_geometry as the active geometry
    first_stops_gdf = first_stops_gdf.drop(columns=['geometry'])
    first_stops_gdf = gpd.GeoDataFrame(first_stops_gdf, geometry='geometry_destination', crs='EPSG:4326')

    last_stops_gdf = last_stops_gdf.drop(columns=['geometry'])
    last_stops_gdf = gpd.GeoDataFrame(last_stops_gdf, geometry='geometry_origin', crs='EPSG:4326')

    return first_stops_gdf, last_stops_gdf

In [27]:
path_to_depots = '/Users/yhe/github_repo/routee-transit/FTA_Depot/Transit_Depot.shp'

first_stops_gdf, last_stops_gdf = add_depot_to_blocks(feed, path_to_depots)

In [28]:
first_stops_gdf

,block_id,arrival_time,stop_lat,stop_lon,nearest_depot_idx,geometry_destination,geometry_origin
0,1153838,0 days 06:24:00,40.659597,-111.895438,738,POINT (-111.89544 40.6596),POINT (-111.91328 40.692613)
1,1153853,0 days 06:22:00,40.772150,-111.933123,746,POINT (-111.93312 40.77215),POINT (-111.909858 40.765876)
2,1153861,0 days 05:51:00,40.772150,-111.933123,746,POINT (-111.93312 40.77215),POINT (-111.909858 40.765876)
3,1153862,0 days 05:21:00,40.772150,-111.933123,746,POINT (-111.93312 40.77215),POINT (-111.909858 40.765876)
4,1153866,0 days 07:30:00,40.761431,-111.888044,1784,POINT (-111.88804 40.76143),POINT (-111.909225 40.764022)
...,...,...,...,...,...,...,...
959,1159879,0 days 15:19:00,40.660163,-111.895228,738,POINT (-111.89523 40.66016),POINT (-111.91328 40.692613)
960,1159880,0 days 06:09:00,40.760491,-111.866141,1784,POINT (-111.86614 40.76049),POINT (-111.909225 40.764022)
961,1159881,0 days 15:09:00,40.760491,-111.866141,1784,POINT (-111.86614 40.76049),POINT (-111.909225 40.764022)
962,1159882,0 days 05:25:00,40.760491,-111.866141,1784,POINT (-111.86614 40.76049),POINT (-111.909225 40.764022)


In [158]:
import pandas as pd
import geopandas as gpd
# single-global-graph approach; per-row cached graphs removed
import multiprocessing as mp
from typing import Tuple, Any

# optional imports for routing/map-matching kept at module level so we don't import per-row
try:
    import osmnx as ox_module
    import networkx as nx_module
except Exception:
    ox_module = None
    nx_module = None

# Optionally hold a single pre-fetched graph for the whole study area.
GLOBAL_GRAPH = None


# Note: per-row cached graph fetch removed. This module now requires a single
# pre-fetched study-area graph assigned to GLOBAL_GRAPH (via bbox in
# `add_deadhead_trips`) and all routing workers use that graph.


def _parallel_map(func, iterable, n_processes: int | None, chunksize: int = 8):
    """Simple wrapper to run func over iterable using multiprocessing when requested."""
    if not n_processes or n_processes <= 1:
        return [func(x) for x in iterable]
    results = []
    with mp.Pool(n_processes) as pool:
        for item in pool.imap(func, iterable, chunksize=chunksize):
            results.append(item)
    return results

def _process_deadhead_trip_row(args: Tuple[Any, ...]):
    """Worker that computes shortest-path and returns per-point shape rows.

    args: (idx, start_x, start_y, end_x, end_y, block_id, network_type, road_buffer_m)

    Returns a list of dicts with fields:
      - 'shape_id' (block id)
      - 'shape_pt_sequence' (1-based int)
      - 'shape_pt_lon'
      - 'shape_pt_lat'
      - 'shape_dist_traveled' (cumulative km from route start)
    """
    from shapely.geometry import LineString
    from shapely.ops import linemerge, unary_union
    import math

    def _haversine_km(lat1, lon1, lat2, lon2):
        R = 6371.0
        phi1 = math.radians(lat1)
        phi2 = math.radians(lat2)
        dphi = math.radians(lat2 - lat1)
        dlambda = math.radians(lon2 - lon1)
        a = math.sin(dphi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2.0) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return R * c

    (
        idx,
        start_x,
        start_y,
        end_x,
        end_y,
        block_id,
        network_type,
        road_buffer_m,
    ) = args

    if ox_module is None or nx_module is None:
        return []

    G = GLOBAL_GRAPH
    if G is None:
        return []

    try:
        src = ox_module.nearest_nodes(G, start_x, start_y)
        dst = ox_module.nearest_nodes(G, end_x, end_y)
        route_nodes = nx_module.shortest_path(G, src, dst, weight='length')

        edge_geoms = []
        for u, v in zip(route_nodes[:-1], route_nodes[1:]):
            data = G.get_edge_data(u, v)
            if not data:
                continue
            key = list(data.keys())[0]
            attr = data[key]
            geom = attr.get('geometry')
            if geom is None:
                ux = G.nodes[u].get('x')
                uy = G.nodes[u].get('y')
                vx = G.nodes[v].get('x')
                vy = G.nodes[v].get('y')
                geom = LineString([(ux, uy), (vx, vy)])
            edge_geoms.append(geom)

        if not edge_geoms: # Return a link from start to end if no route found
            return [{
                'shape_id': block_id,
                'shape_pt_sequence': 1,
                'shape_pt_lon': float(start_x),
                'shape_pt_lat': float(start_y),
                'shape_dist_traveled': 0.0,
            }, {
                'shape_id': block_id,
                'shape_pt_sequence': 2,
                'shape_pt_lon': float(end_x),
                'shape_pt_lat': float(end_y),
                'shape_dist_traveled': _haversine_km(start_y, start_x, end_y, end_x),
            }]

        try:
            merged = linemerge(edge_geoms)
        except Exception:
            merged = unary_union(edge_geoms)
        if getattr(merged, 'geom_type', '') == 'MultiLineString':
            coords = []
            for part in merged:
                coords.extend(list(part.coords))
            merged = LineString(coords)

        coords = list(merged.coords)  # list of (lon, lat)
        rows = []
        prev_lat = None
        prev_lon = None
        cum_km = 0.0
        for seq, (lon, lat) in enumerate(coords, start=1):
            if prev_lat is not None:
                seg_km = _haversine_km(prev_lat, prev_lon, lat, lon)
                cum_km += seg_km
            else:
                cum_km = 0.0
            rows.append({
                'shape_id': block_id,
                'shape_pt_sequence': int(seq),
                'shape_pt_lon': float(lon),
                'shape_pt_lat': float(lat),
                'shape_dist_traveled': float(cum_km),
            })
            prev_lat, prev_lon = lat, lon

        return rows
    except Exception:
        return []


def add_deadhead_trips(
    df: gpd.GeoDataFrame,
    o_col: str = 'geometry_origin',
    d_col: str = 'geometry_destination',
    network_type: str = 'drive',
    road_buffer_m: int = 2000,
    n_processes: int | None = None,
    bbox: tuple[float, float, float, float] | None = None,
    ) -> pd.DataFrame:
    """Compute deadhead route shapes between origin and destination.

    For each row in `df` this computes a shortest-path on the OSM network between the
    origin (o_col) and destination (d_col) using a single pre-fetched study-area graph
    (fetched from `bbox`). Returns a pandas DataFrame with per-point GTFS-like shape
    rows: ['shape_id', 'shape_pt_sequence', 'shape_pt_lon', 'shape_pt_lat', 'shape_dist_traveled']
    where `shape_dist_traveled` is cumulative distance in kilometers from the route start.
    """
    # Require bbox: this function uses a single study-area graph for all routing.
    global GLOBAL_GRAPH
    if bbox is None:
        raise ValueError('bbox is required: provide a study-area bounding box (minx, miny, maxx, maxy)')
    if ox_module is None:
        raise ImportError('osmnx is required to fetch a global bbox graph')
    # bbox expected as (minx, miny, maxx, maxy)
    try:
        minx, miny, maxx, maxy = bbox
    except Exception:
        raise ValueError('bbox must be a 4-tuple (minx, miny, maxx, maxy)')
    # ox.graph_from_bbox signature: (north, south, east, west)
    GLOBAL_GRAPH = ox_module.graph_from_bbox(bbox, network_type='drive')
    
    task_args = []
    for idx, r in df.iterrows():
        origin = r.get(o_col)
        destination = r.get(d_col)
        if origin is None or destination is None:
            continue
        task_args.append((
            idx,
            float(origin.x),
            float(origin.y),
            float(destination.x),
            float(destination.y),
            r.get('block_id') if 'block_id' in r else None,
            network_type,
            road_buffer_m,
        ))

    results = _parallel_map(_process_deadhead_trip_row, task_args, n_processes)

    # results is a list of lists (per-route rows). Flatten and build DataFrame
    shape_rows = []
    for route_rows in results:
        if not route_rows:
            continue
        # route_rows is a list of dicts from the worker
        shape_rows.extend(route_rows)

    out_df = pd.DataFrame(
        shape_rows,
        columns=['shape_id', 'shape_pt_sequence', 'shape_pt_lon', 'shape_pt_lat', 'shape_dist_traveled'],
    )
    return out_df



In [31]:
# Combine all start and end points
all_points = pd.concat([first_stops_gdf['geometry_origin'], first_stops_gdf['geometry_destination']])

# Extract lon/lat lists
lons = all_points.apply(lambda p: p.x)
lats = all_points.apply(lambda p: p.y)

# Bounding box
min_lon, max_lon = lons.min(), lons.max()
min_lat, max_lat = lats.min(), lats.max()

print(min_lon, min_lat, max_lon, max_lat)

# Roughly 2 km buffer in degrees
buffer_deg_lat = 0.018
buffer_deg_lon = 0.022

miny = min_lat - buffer_deg_lat
maxy = max_lat + buffer_deg_lat
minx = min_lon - buffer_deg_lon
maxx = max_lon + buffer_deg_lon


result_df = add_deadhead_trips(
    df = first_stops_gdf[0:4],
    n_processes = 1,
    bbox = [minx, miny, maxx, maxy ]
    )

-112.340301 39.97639 -111.504474 41.742741988


In [32]:
result_df

,shape_id,shape_pt_sequence,shape_pt_lon,shape_pt_lat,shape_dist_traveled
0,1153838,1,-111.911194,40.691679,0.000000
1,1153838,2,-111.911192,40.691096,0.064860
2,1153838,3,-111.911192,40.690962,0.079671
3,1153838,4,-111.911193,40.690444,0.137282
4,1153838,5,-111.911193,40.690361,0.146522
...,...,...,...,...,...
481,1153862,88,-111.931909,40.771634,2.606374
482,1153862,89,-111.932192,40.771635,2.630214
483,1153862,90,-111.932871,40.771636,2.687401
484,1153862,91,-111.932928,40.771636,2.692201


In [12]:
bbox = [minx, miny, maxx, maxy ]
# Require bbox: this function uses a single study-area graph for all routing.
global GLOBAL_GRAPH
if bbox is None:
    raise ValueError('bbox is required: provide a study-area bounding box (minx, miny, maxx, maxy)')
if ox_module is None:
    raise ImportError('osmnx is required to fetch a global bbox graph')
# bbox expected as (minx, miny, maxx, maxy)
try:
    minx, miny, maxx, maxy = bbox
except Exception:
    raise ValueError('bbox must be a 4-tuple (minx, miny, maxx, maxy)')
# ox.graph_from_bbox signature: (north, south, east, west)
GLOBAL_GRAPH = ox_module.graph_from_bbox(bbox, network_type='drive')

In [13]:
GLOBAL_GRAPH

In [15]:
df = first_stops_gdf
o_col = 'geometry_origin'
d_col = 'geometry_destination'
network_type = 'drive'
road_buffer_m = 2000

task_args = []
for idx, r in df.iterrows():
    origin = r.get(o_col)
    print(origin)
    destination = r.get(d_col)
    print(origin)
    if origin is None or destination is None:
        continue
    task_args.append((
        idx,
        float(origin.x),
        float(origin.y),
        float(destination.x),
        float(destination.y),
        r.get('block_id') if 'block_id' in r else None,
        network_type,
        road_buffer_m,
    ))

POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.909858 40.765876)
POINT (-111.909858 40.765876)
POINT (-111.909858 40.765876)
POINT (-111.909858 40.765876)
POINT (-111.909858 40.765876)
POINT (-111.909858 40.765876)
POINT (-111.909225 40.764022)
POINT (-111.909225 40.764022)
POINT (-111.911252 40.762413)
POINT (-111.911252 40.762413)
POINT (-111.911252 40.762413)
POINT (-111.911252 40.762413)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.911252 40.762413)
POINT (-111.911252 40.762413)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.911252 40.762413)
POINT (-111.911252 40.762413)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613)
POINT (-111.91328 40.692613

In [144]:
df_test = df[df.block_id == '1156055']

In [159]:
check_df = add_deadhead_trips(
    df = df_test,
    n_processes = 1,
    bbox = [minx, miny, maxx, maxy]
    )

#add_deadhead_trips

In [19]:
src = ox_module.nearest_nodes(G, start_x, start_y)
dst = ox_module.nearest_nodes(G, end_x, end_y)
route_nodes = nx_module.shortest_path(G, src, dst, weight='length')

edge_geoms = []
for u, v in zip(route_nodes[:-1], route_nodes[1:]):
    data = G.get_edge_data(u, v)
    if not data:
        continue
    key = list(data.keys())[0]
    attr = data[key]
    geom = attr.get('geometry')
    if geom is None:
        ux = G.nodes[u].get('x')
        uy = G.nodes[u].get('y')
        vx = G.nodes[v].get('x')
        vy = G.nodes[v].get('y')
        geom = LineString([(ux, uy), (vx, vy)])
    edge_geoms.append(geom)

In [20]:
edge_geoms

[<LINESTRING (-111.911 40.692, -111.911 40.691, -111.911 40.691, -111.911 40....>,
 <LINESTRING (-111.911 40.69, -111.911 40.69, -111.911 40.69, -111.911 40.69)>,
 <LINESTRING (-111.911 40.69, -111.911 40.69, -111.911 40.69, -111.911 40.69,...>,
 <LINESTRING (-111.91 40.687, -111.91 40.687)>,
 <LINESTRING (-111.91 40.687, -111.909 40.687, -111.909 40.687)>,
 <LINESTRING (-111.909 40.687, -111.908 40.687, -111.908 40.687)>,
 <LINESTRING (-111.908 40.687, -111.907 40.687, -111.907 40.687, -111.906 40....>,
 <LINESTRING (-111.906 40.687, -111.905 40.687, -111.905 40.687, -111.905 40....>,
 <LINESTRING (-111.902 40.687, -111.902 40.687, -111.902 40.687, -111.902 40....>,
 <LINESTRING (-111.902 40.686, -111.902 40.686, -111.901 40.686, -111.901 40....>,
 <LINESTRING (-111.9 40.682, -111.9 40.682, -111.9 40.681, -111.9 40.681, -11...>,
 <LINESTRING (-111.9 40.676, -111.9 40.676, -111.899 40.676, -111.899 40.676)>,
 <LINESTRING (-111.899 40.676, -111.899 40.676, -111.899 40.676, -111.899 40..

In [108]:
trips_df = feed.trips
stop_times_df = feed.stop_times
stops_df = feed.stops   
blocks_trips_stops = stop_times_df.merge(trips_df[['trip_id', 'block_id']], on='trip_id', how='left')
blocks_trips_stops = blocks_trips_stops.merge(stops_df, on='stop_id', how='left')

blocks_trips_stops = blocks_trips_stops.sort_values(by=['block_id', 'arrival_time'])
first_stops = blocks_trips_stops.groupby('block_id').first().reset_index()
last_stops = blocks_trips_stops.groupby('block_id').last().reset_index()

In [111]:
first_stops = first_stops[['block_id', 'arrival_time', 'stop_lat', 'stop_lon']]
last_stops = last_stops[['block_id', 'arrival_time', 'stop_lat', 'stop_lon']]

first_stops['geometry'] = first_stops.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
last_stops['geometry'] = last_stops.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
first_stops_gdf = gpd.GeoDataFrame(first_stops, geometry='geometry', crs='EPSG:4326')
last_stops_gdf = gpd.GeoDataFrame(last_stops, geometry='geometry', crs='EPSG:4326')

# Read in depot locations
depots_df = gpd.read_file('/Users/yhe/github_repo/routee-transit/FTA_Depot/Transit_Depot.shp')
depots_df = depots_df.to_crs(epsg=4326)

# Find the nearest depot to each first and last stop
first_stops_gdf['nearest_depot'] = first_stops_gdf.geometry.apply(
    lambda x: depots_df.distance(x).idxmin()
)
last_stops_gdf['nearest_depot'] = last_stops_gdf.geometry.apply(
    lambda x: depots_df.distance(x).idxmin()
)

/var/folders/31/y4mwl1qs14z8r1x7g524813cc7rklk/T/ipykernel_95751/3389172204.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lambda x: depots_df.distance(x).idxmin()
/var/folders/31/y4mwl1qs14z8r1x7g524813cc7rklk/T/ipykernel_95751/3389172204.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lambda x: depots_df.distance(x).idxmin()
/var/folders/31/y4mwl1qs14z8r1x7g524813cc7rklk/T/ipykernel_95751/3389172204.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lambda x: depots_df.distance(x).idxmin()
/var/folders/31/y4mwl1qs14z8r1x7g524813cc7rklk/T/ipykernel_95751/33

In [114]:
first_stops_gdf = first_stops_gdf.merge(
        depots_df[['geometry']], left_on='nearest_depot', right_index=True, suffixes=('_stop', '_depot')            
    )
last_stops_gdf = last_stops_gdf.merge(
        depots_df[['geometry']], left_on='nearest_depot', right_index=True, suffixes=('_stop', '_depot')            
    )

In [140]:
first_stops_gdf

,block_id,arrival_time,stop_lat,stop_lon,geometry_stop,nearest_depot,geometry_depot
0,1153838,0 days 06:24:00,40.659597,-111.895438,POINT (-111.89544 40.6596),738,POINT (-111.91328 40.69261)
1,1153853,0 days 06:22:00,40.772150,-111.933123,POINT (-111.93312 40.77215),753,POINT (-111.91125 40.76241)
2,1153861,0 days 05:51:00,40.772150,-111.933123,POINT (-111.93312 40.77215),753,POINT (-111.91125 40.76241)
3,1153862,0 days 05:21:00,40.772150,-111.933123,POINT (-111.93312 40.77215),753,POINT (-111.91125 40.76241)
4,1153866,0 days 07:30:00,40.761431,-111.888044,POINT (-111.88804 40.76143),749,POINT (-111.90922 40.76402)
...,...,...,...,...,...,...,...
959,1159879,0 days 15:19:00,40.660163,-111.895228,POINT (-111.89523 40.66016),738,POINT (-111.91328 40.69261)
960,1159880,0 days 06:09:00,40.760491,-111.866141,POINT (-111.86614 40.76049),749,POINT (-111.90922 40.76402)
961,1159881,0 days 15:09:00,40.760491,-111.866141,POINT (-111.86614 40.76049),749,POINT (-111.90922 40.76402)
962,1159882,0 days 05:25:00,40.760491,-111.866141,POINT (-111.86614 40.76049),749,POINT (-111.90922 40.76402)


In [142]:
test_df = first_stops_gdf[first_stops_gdf['block_id']=='1153838']

In [143]:
test_df

,block_id,arrival_time,stop_lat,stop_lon,geometry_stop,nearest_depot,geometry_depot
0,1153838,0 days 06:24:00,40.659597,-111.895438,POINT (-111.89544 40.6596),738,POINT (-111.91328 40.69261)


In [135]:
test["actual_time"] = test["actual_time"].astype(str)

test.to_file('/Users/yhe/Downloads/test.geojson', driver="GeoJSON")

In [14]:
trips = gtfs_slc.trips

In [18]:
trips

,trip_id,route_id,service_id,block_id,shape_id
0,5182559,19930,2,1155937,226296
1,5182560,19930,2,1155939,226296
2,5182561,19930,2,1155941,226296
3,5182562,19930,2,1155938,226296
4,5182563,19930,2,1155943,226296
...,...,...,...,...,...
14902,5206452,91962,22,1157708,227290
14903,5206453,91962,22,1157709,227290
14904,5206454,91962,22,1157708,227290
14905,5206455,91962,22,1157709,227290


In [20]:
len(trips.block_id.unique())

964

In [23]:
stop_times = gtfs_slc.stop_times

In [25]:
stop_times

,trip_id,stop_sequence,arrival_time,stop_id
0,5182559,1,0 days 05:45:00,5502
1,5182559,2,0 days 05:46:05,13095
2,5182559,3,0 days 05:47:00,5504
3,5182559,4,0 days 05:47:31,13192
4,5182559,5,0 days 05:48:43,21929
...,...,...,...,...
602361,5206456,4,0 days 23:31:15,15669
602362,5206456,5,0 days 23:35:00,20381
602363,5206456,6,0 days 23:37:46,19178
602364,5206456,7,0 days 23:40:30,24429


In [24]:
stops = gtfs_slc.stops

In [106]:
stops

,stop_id,stop_lat,stop_lon
0,391,40.663345,-111.987053
1,506,40.660280,-111.987056
2,870,40.692636,-111.958136
3,887,40.693910,-111.957789
4,1022,40.772535,-111.837594
...,...,...,...
5229,25407,40.718417,-111.983917
5230,25410,40.580596,-111.829459
5231,25421,41.094656,-112.013359
5232,25423,40.983302,-111.911566


In [26]:
blocks_trips_stops = stop_times.merge(trips, on = 'trip_id', how = 'left')
blocks_trips_stops

,trip_id,stop_sequence,arrival_time,stop_id,route_id,service_id,block_id,shape_id
0,5182559,1,0 days 05:45:00,5502,19930,2,1155937,226296
1,5182559,2,0 days 05:46:05,13095,19930,2,1155937,226296
2,5182559,3,0 days 05:47:00,5504,19930,2,1155937,226296
3,5182559,4,0 days 05:47:31,13192,19930,2,1155937,226296
4,5182559,5,0 days 05:48:43,21929,19930,2,1155937,226296
...,...,...,...,...,...,...,...,...
602361,5206456,4,0 days 23:31:15,15669,91962,22,1157708,227290
602362,5206456,5,0 days 23:35:00,20381,91962,22,1157708,227290
602363,5206456,6,0 days 23:37:46,19178,91962,22,1157708,227290
602364,5206456,7,0 days 23:40:30,24429,91962,22,1157708,227290


In [27]:
blocks_trips_stops_gps = blocks_trips_stops.merge(stops, on = 'stop_id', how = 'left')
blocks_trips_stops_gps

,trip_id,stop_sequence,arrival_time,stop_id,route_id,service_id,block_id,shape_id,stop_lat,stop_lon
0,5182559,1,0 days 05:45:00,5502,19930,2,1155937,226296,40.761431,-111.888044
1,5182559,2,0 days 05:46:05,13095,19930,2,1155937,226296,40.763556,-111.888072
2,5182559,3,0 days 05:47:00,5504,19930,2,1155937,226296,40.765870,-111.888062
3,5182559,4,0 days 05:47:31,13192,19930,2,1155937,226296,40.767803,-111.888086
4,5182559,5,0 days 05:48:43,21929,19930,2,1155937,226296,40.771666,-111.889348
...,...,...,...,...,...,...,...,...,...,...
602361,5206456,4,0 days 23:31:15,15669,91962,22,1157708,227290,40.763120,-111.835721
602362,5206456,5,0 days 23:35:00,20381,91962,22,1157708,227290,40.770008,-111.839202
602363,5206456,6,0 days 23:37:46,19178,91962,22,1157708,227290,40.761201,-111.838088
602364,5206456,7,0 days 23:40:30,24429,91962,22,1157708,227290,40.760730,-111.851183


In [50]:
# get the first stop and last stop of each block
blocks_trips_stops_gps = blocks_trips_stops_gps.sort_values(by = (['block_id', 'arrival_time']))
blocks = blocks_trips_stops_gps.groupby('block_id').agg({'trip_id': ['first', 'last'],'arrival_time': ['first', 'last'],
                                                'stop_id': ['first', 'last'], 'stop_lat': ['first', 'last'],
                                               'stop_lon': ['first', 'last']}).reset_index()

In [51]:
blocks

block_id  trip_id             arrival_time                 stop_id         \
                first     last           first            last   first   last   
0    1153838  5171658  5171706 0 days 06:24:00 0 days 19:11:00   18195  24631   
1    1153853  5167714  5168779 0 days 06:22:00 0 days 20:13:00   24044  24379   
2    1153861  5167721  5173960 0 days 05:51:00 0 days 19:25:00   24044  22287   
3    1153862  5167723  5174097 0 days 05:21:00 0 days 18:57:00   24044  21502   
4    1153866  5167477  5167512 0 days 07:30:00 0 days 11:36:00    5502  14598   
..       ...      ...      ...             ...             ...     ...    ...   
959  1159879  5203928  5203962 0 days 15:19:00 1 days 00:09:00   18061  18061   
960  1159880  5231624  5231641 0 days 06:09:00 0 days 14:48:00   18262  18262   
961  1159881  5231642  5231658 0 days 15:09:00 0 days 23:18:00   18262  18262   
962  1159882  5231659  5231659 0 days 05:25:00 0 days 09:35:00   18262  18262   
963  1159883  5231660  5231660 0 days 16:25:00 0 days 19:35:00   18262  18262   

      stop_lat               stop_lon              
         first       last       first        last  
0    40.659597  40.774009 -111.895438 -111.904983  
1    40.772150  40.763678 -111.933123 -111.909788  
2    40.772150  40.693438 -111.933123 -111.960025  
3    40.772150  40.638454 -111.933123 -112.027011  
4    40.761431  40.760010 -111.888044 -111.888422  
..         ...        ...         ...         ...  
959  40.660163  40.660163 -111.895228 -111.895228  
960  40.760491  40.760491 -111.866141 -111.866141  
961  40.760491  40.760491 -111.866141 -111.866141  
962  40.760491  40.760491 -111.866141 -111.866141  
963  40.760491  40.760491 -111.866141 -111.866141  

[964 rows x 11 columns]

In [52]:
blocks.columns = ['block_id', 'trip_id_first', 'trip_id_last', 'arrival_time_first', 'arrival_time_last',
                 'stop_id_first', 'stop_id_last', 'stop_lat_first', 'stop_lat_last', 'stop_lon_first', 'stop_lon_last']

In [82]:
blocks

,block_id,trip_id_first,trip_id_last,arrival_time_first,arrival_time_last,stop_id_first,stop_id_last,stop_lat_first,stop_lat_last,stop_lon_first,stop_lon_last,stop_coords_first,stop_coords_last
0,1153838,5171658,5171706,0 days 06:24:00,0 days 19:11:00,18195,24631,40.659597,40.774009,-111.895438,-111.904983,POINT (-111.895438 40.659597),POINT (-111.904983 40.774009)
1,1153853,5167714,5168779,0 days 06:22:00,0 days 20:13:00,24044,24379,40.772150,40.763678,-111.933123,-111.909788,POINT (-111.933123 40.77215),POINT (-111.909788 40.763678)
2,1153861,5167721,5173960,0 days 05:51:00,0 days 19:25:00,24044,22287,40.772150,40.693438,-111.933123,-111.960025,POINT (-111.933123 40.77215),POINT (-111.960025 40.693438)
3,1153862,5167723,5174097,0 days 05:21:00,0 days 18:57:00,24044,21502,40.772150,40.638454,-111.933123,-112.027011,POINT (-111.933123 40.77215),POINT (-112.027011 40.638454)
4,1153866,5167477,5167512,0 days 07:30:00,0 days 11:36:00,5502,14598,40.761431,40.760010,-111.888044,-111.888422,POINT (-111.888044 40.761431),POINT (-111.888422 40.76001)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,1159879,5203928,5203962,0 days 15:19:00,1 days 00:09:00,18061,18061,40.660163,40.660163,-111.895228,-111.895228,POINT (-111.895228 40.660163),POINT (-111.895228 40.660163)
960,1159880,5231624,5231641,0 days 06:09:00,0 days 14:48:00,18262,18262,40.760491,40.760491,-111.866141,-111.866141,POINT (-111.866141 40.760491),POINT (-111.866141 40.760491)
961,1159881,5231642,5231658,0 days 15:09:00,0 days 23:18:00,18262,18262,40.760491,40.760491,-111.866141,-111.866141,POINT (-111.866141 40.760491),POINT (-111.866141 40.760491)
962,1159882,5231659,5231659,0 days 05:25:00,0 days 09:35:00,18262,18262,40.760491,40.760491,-111.866141,-111.866141,POINT (-111.866141 40.760491),POINT (-111.866141 40.760491)


In [83]:
blocks["stop_coords_first"] = blocks.apply(lambda row: Point(row["stop_lon_first"], row["stop_lat_first"]), axis=1)
blocks["stop_coords_last"] = blocks.apply(lambda row: Point(row["stop_lon_last"], row["stop_lat_last"]), axis=1)

In [84]:
blocks

,block_id,trip_id_first,trip_id_last,arrival_time_first,arrival_time_last,stop_id_first,stop_id_last,stop_lat_first,stop_lat_last,stop_lon_first,stop_lon_last,stop_coords_first,stop_coords_last
0,1153838,5171658,5171706,0 days 06:24:00,0 days 19:11:00,18195,24631,40.659597,40.774009,-111.895438,-111.904983,POINT (-111.895438 40.659597),POINT (-111.904983 40.774009)
1,1153853,5167714,5168779,0 days 06:22:00,0 days 20:13:00,24044,24379,40.772150,40.763678,-111.933123,-111.909788,POINT (-111.933123 40.77215),POINT (-111.909788 40.763678)
2,1153861,5167721,5173960,0 days 05:51:00,0 days 19:25:00,24044,22287,40.772150,40.693438,-111.933123,-111.960025,POINT (-111.933123 40.77215),POINT (-111.960025 40.693438)
3,1153862,5167723,5174097,0 days 05:21:00,0 days 18:57:00,24044,21502,40.772150,40.638454,-111.933123,-112.027011,POINT (-111.933123 40.77215),POINT (-112.027011 40.638454)
4,1153866,5167477,5167512,0 days 07:30:00,0 days 11:36:00,5502,14598,40.761431,40.760010,-111.888044,-111.888422,POINT (-111.888044 40.761431),POINT (-111.888422 40.76001)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,1159879,5203928,5203962,0 days 15:19:00,1 days 00:09:00,18061,18061,40.660163,40.660163,-111.895228,-111.895228,POINT (-111.895228 40.660163),POINT (-111.895228 40.660163)
960,1159880,5231624,5231641,0 days 06:09:00,0 days 14:48:00,18262,18262,40.760491,40.760491,-111.866141,-111.866141,POINT (-111.866141 40.760491),POINT (-111.866141 40.760491)
961,1159881,5231642,5231658,0 days 15:09:00,0 days 23:18:00,18262,18262,40.760491,40.760491,-111.866141,-111.866141,POINT (-111.866141 40.760491),POINT (-111.866141 40.760491)
962,1159882,5231659,5231659,0 days 05:25:00,0 days 09:35:00,18262,18262,40.760491,40.760491,-111.866141,-111.866141,POINT (-111.866141 40.760491),POINT (-111.866141 40.760491)


In [85]:
blocks_first_stop = blocks[['block_id', 'stop_coords_first']]
blocks_last_stop = blocks[['block_id', 'stop_coords_last']]

In [86]:
blocks_last_stop

,block_id,stop_coords_last
0,1153838,POINT (-111.904983 40.774009)
1,1153853,POINT (-111.909788 40.763678)
2,1153861,POINT (-111.960025 40.693438)
3,1153862,POINT (-112.027011 40.638454)
4,1153866,POINT (-111.888422 40.76001)
...,...,...
959,1159879,POINT (-111.895228 40.660163)
960,1159880,POINT (-111.866141 40.760491)
961,1159881,POINT (-111.866141 40.760491)
962,1159882,POINT (-111.866141 40.760491)


In [87]:
blocks_first_stop_gdf = gpd.GeoDataFrame(blocks_first_stop, geometry = 'stop_coords_first')
blocks_last_stop_gdf = gpd.GeoDataFrame(blocks_last_stop, geometry = 'stop_coords_last')

In [88]:
blocks_first_stop_gdf

,block_id,stop_coords_first
0,1153838,POINT (-111.89544 40.6596)
1,1153853,POINT (-111.93312 40.77215)
2,1153861,POINT (-111.93312 40.77215)
3,1153862,POINT (-111.93312 40.77215)
4,1153866,POINT (-111.88804 40.76143)
...,...,...
959,1159879,POINT (-111.89523 40.66016)
960,1159880,POINT (-111.86614 40.76049)
961,1159881,POINT (-111.86614 40.76049)
962,1159882,POINT (-111.86614 40.76049)


In [89]:
transit_depots = gpd.read_file('/Users/yhe/github_repo/routee-transit/FTA_Depot/Transit_Depot.shp')

In [90]:
transit_depots

,Facility T,State,NTD ID,geometry
0,General Purpose Maintenance Facility/Depot,WA,00001,POINT (-122.3219 47.58858)
1,Maintenance Facility (Service and Inspection),WA,00001,POINT (-122.3228 47.58876)
2,Maintenance Facility (Service and Inspection),WA,00001,POINT (-122.1725 47.62617)
3,Maintenance Facility (Service and Inspection),WA,00001,POINT (-122.3242 47.58883)
4,Maintenance Facility (Service and Inspection),WA,00001,POINT (-122.1788 47.62722)
...,...,...,...,...
2558,Transit Agency Headquarters,AZ,99433,POINT (-113.41988 35.5385)
2559,Transit Agency Headquarters,AZ,99436,POINT (-111.76035 33.06944)
2560,Transit Agency Headquarters,CA,99443,POINT (-118.04619 34.04458)
2561,Transit Agency Headquarters,CA,99450,POINT (-118.08232 33.82933)


In [91]:
transit_depots[transit_depots['State']=='UT']

,Facility T,State,NTD ID,geometry
737,General Purpose Maintenance Facility/Depot,UT,80001,POINT (-111.90908 40.76555)
738,General Purpose Maintenance Facility/Depot,UT,80001,POINT (-111.91328 40.69261)
739,General Purpose Maintenance Facility/Depot,UT,80001,POINT (-111.98024 41.23607)
740,General Purpose Maintenance Facility/Depot,UT,80001,POINT (-111.9183 40.69212)
741,General Purpose Maintenance Facility/Depot,UT,80001,POINT (-111.73097 40.27722)
742,General Purpose Maintenance Facility/Depot,UT,80001,POINT (-111.9091 40.76555)
743,General Purpose Maintenance Facility/Depot,UT,80001,POINT (-111.73168 40.27726)
744,General Purpose Maintenance Facility/Depot,UT,80001,POINT (-111.73232 40.27738)
745,General Purpose Maintenance Facility/Depot,UT,80001,POINT (-112.3403 40.52976)
746,General Purpose Maintenance Facility/Depot,UT,80001,POINT (-111.90986 40.76588)


In [92]:
blocks_first_stop_gdf = blocks_first_stop_gdf.set_crs(epsg=4326)
transit_depots = transit_depots.set_crs(epsg=4326)
blocks_first_stop_gdf = blocks_first_stop_gdf.to_crs(epsg=3857)
transit_depots = transit_depots.to_crs(epsg=3857)

In [100]:
blocks_first_stop_nearest = gpd.sjoin_nearest(
    blocks_first_stop_gdf, transit_depots, how="left", distance_col="dist_to_nearest"
)

In [101]:
blocks_first_stop_nearest

,block_id,stop_coords_first,index_right,Facility T,State,NTD ID,dist_to_nearest
0,1153838,POINT (-12456143.18 4962261.216),738,General Purpose Maintenance Facility/Depot,UT,80001,5237.335179
0,1153838,POINT (-12456143.18 4962261.216),1773,General Purpose Maintenance Facility/Depot,UT,80001,5237.335179
1,1153853,POINT (-12460338.255 4978791.709),746,General Purpose Maintenance Facility/Depot,UT,80001,2749.135396
1,1153853,POINT (-12460338.255 4978791.709),1781,General Purpose Maintenance Facility/Depot,UT,80001,2749.135396
2,1153861,POINT (-12460338.255 4978791.709),746,General Purpose Maintenance Facility/Depot,UT,80001,2749.135396
...,...,...,...,...,...,...,...
961,1159881,POINT (-12452881.853 4977078.07),749,General Purpose Maintenance Facility/Depot,UT,80001,4824.083611
962,1159882,POINT (-12452881.853 4977078.07),1784,General Purpose Maintenance Facility/Depot,UT,80001,4824.083611
962,1159882,POINT (-12452881.853 4977078.07),749,General Purpose Maintenance Facility/Depot,UT,80001,4824.083611
963,1159883,POINT (-12452881.853 4977078.07),1784,General Purpose Maintenance Facility/Depot,UT,80001,4824.083611


In [102]:
blocks_first_stop_nearest = blocks_first_stop_nearest.assign(
    depot=transit_depots.loc[blocks_first_stop_nearest["index_right"], "geometry"].values
)

In [103]:
blocks_first_stop_nearest

,block_id,stop_coords_first,index_right,Facility T,State,NTD ID,dist_to_nearest,depot
0,1153838,POINT (-12456143.18 4962261.216),738,General Purpose Maintenance Facility/Depot,UT,80001,5237.335179,POINT (-12458129.343 4967107.332)
0,1153838,POINT (-12456143.18 4962261.216),1773,General Purpose Maintenance Facility/Depot,UT,80001,5237.335179,POINT (-12458129.343 4967107.332)
1,1153853,POINT (-12460338.255 4978791.709),746,General Purpose Maintenance Facility/Depot,UT,80001,2749.135396,POINT (-12457748.407 4977869.52)
1,1153853,POINT (-12460338.255 4978791.709),1781,General Purpose Maintenance Facility/Depot,UT,80001,2749.135396,POINT (-12457748.407 4977869.52)
2,1153861,POINT (-12460338.255 4978791.709),746,General Purpose Maintenance Facility/Depot,UT,80001,2749.135396,POINT (-12457748.407 4977869.52)
...,...,...,...,...,...,...,...,...
961,1159881,POINT (-12452881.853 4977078.07),749,General Purpose Maintenance Facility/Depot,UT,80001,4824.083611,POINT (-12457677.942 4977597.025)
962,1159882,POINT (-12452881.853 4977078.07),1784,General Purpose Maintenance Facility/Depot,UT,80001,4824.083611,POINT (-12457677.942 4977597.025)
962,1159882,POINT (-12452881.853 4977078.07),749,General Purpose Maintenance Facility/Depot,UT,80001,4824.083611,POINT (-12457677.942 4977597.025)
963,1159883,POINT (-12452881.853 4977078.07),1784,General Purpose Maintenance Facility/Depot,UT,80001,4824.083611,POINT (-12457677.942 4977597.025)


In [104]:
blocks_first_stop_nearest = blocks_first_stop_nearest.drop_duplicates(subset='block_id', keep='first')
blocks_first_stop_nearest = blocks_first_stop_nearest[['block_id', 'stop_coords_first', 'depot', 'dist_to_nearest']]

In [105]:
blocks_first_stop_nearest

,block_id,stop_coords_first,depot,dist_to_nearest
0,1153838,POINT (-12456143.18 4962261.216),POINT (-12458129.343 4967107.332),5237.335179
1,1153853,POINT (-12460338.255 4978791.709),POINT (-12457748.407 4977869.52),2749.135396
2,1153861,POINT (-12460338.255 4978791.709),POINT (-12457748.407 4977869.52),2749.135396
3,1153862,POINT (-12460338.255 4978791.709),POINT (-12457748.407 4977869.52),2749.135396
4,1153866,POINT (-12455320.084 4977216.22),POINT (-12457677.942 4977597.025),2388.410960
...,...,...,...,...
959,1159879,POINT (-12456119.803 4962344.273),POINT (-12458129.343 4967107.332),5169.620047
960,1159880,POINT (-12452881.853 4977078.07),POINT (-12457677.942 4977597.025),4824.083611
961,1159881,POINT (-12452881.853 4977078.07),POINT (-12457677.942 4977597.025),4824.083611
962,1159882,POINT (-12452881.853 4977078.07),POINT (-12457677.942 4977597.025),4824.083611
